In [ ]:
from importlib import reload
#reload(Utilities)
#reload(clm)

import sys, os
import re

import pandas as pd
import numpy as np
from pandas.api.types import is_numeric_dtype
from pandas.api.types import is_timedelta64_dtype
from scipy import stats
import datetime
import time
from natsort import natsorted, ns
from packaging import version_
import itertools
from dateutil.parser import parse
from operator import itemgetter

from pmdarima import auto_arima
import statsmodels.api as sm
from statsmodels.tsa.stattools import acovf, acf, pacf, pacf_yw, pacf_ols
from pandas.plotting import lag_plot
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.graphics.tsaplots import month_plot, quarter_plot, seasonal_plot
from statsmodels.tsa.arima_model import ARMA, ARIMA, ARMAResults, ARIMAResults
from statsmodels.tsa.statespace.sarimax import SARIMAX

from arch import arch_model

from scipy.stats.mstats import trim

#---------------------------------------------------------------------
import pyodbc
#---------------------------------------------------------------------
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D
import matplotlib.ticker as ticker
from matplotlib import dates
# import constants for the days of the week
from matplotlib.dates import MO, TU, WE, TH, FR, SA, SU
#---------------------------------------------------------------------
sys.path.insert(0, os.path.realpath('..'))
import Utilities_config
#-----
import CommonLearningMethods as clm
#-----
from AMINonVee import AMINonVee
#---------------------------------------------------------------------
sys.path.insert(0, Utilities_config.get_utilities_dir())
import Utilities
import Utilities_df
import Utilities_dt
import Plot_Box_sns
import GrubbsTest
import DickeyFullerTest as dft

In [ ]:
import warnings
warnings.filterwarnings('ignore')

# -----------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------

In [ ]:
def find_missing_time_idxs(df, freq):
    if df.index.freq is not None:
        assert(freq==df.index.freq)
    expected = pd.date_range(start=df.index[0], end=df.index[-1], freq=freq)
    observed = df.index
    missing = set(expected)-set(observed)
    return missing

def find_non_quarter_hour_entries(time_list):
    time_list = pd.to_datetime(time_list)
    found_abnormal=[]
    for i,time in enumerate(time_list):
        if time.minute % 15 != 0:
            found_abnormal.append((i,time))
        elif time.second != 0:
            found_abnormal.append((i,time))
        else:
            continue
    return found_abnormal

# TODO !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# def find_unexpected_period_lengths(df, start_col, end_col):

# -----------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------

In [ ]:
# read_df = pd.read_csv(os.path.join(Utilities.get_local_data_dir(), r'sample_outages\outg_rec_nb_11751094\outg_rec_nb_11751094_2019_q4.csv'))
# read_df = read_df[read_df['serialnumber']==880687439]
# read_df = read_df[(read_df['aep_derived_uom']=='KWH') & (read_df['aep_srvc_qlty_idntfr']=='TOTAL')]
# read_df = read_df[['starttimeperiod', 'endtimeperiod','aep_endtime_utc', 'timezoneoffset']]
# read_df = read_df.sort_values(by='aep_endtime_utc', ignore_index=True)
# read_df

# -----------------------------------------------------------------------------------------
# -----------------------------------------------------------------------------------------

In [ ]:
# TODO Investigate why annual kwh is not constant!

In [ ]:
fig_num = 0

In [ ]:
cols_of_interest = [
    'serialnumber',
    'prem_nb', 
    'srvc_pole_nb', 
    'trsf_pole_nb',
    'annual_kwh',
    'station_nb',
    'xfmr_nb', 
    'starttimeperiod', 
    'endtimeperiod', 
    'aep_endtime_utc', 
    'timezoneoffset', 
    'aep_derived_uom',
    'aep_srvc_qlty_idntfr', 
    'value', 
    'aep_usage_dt'
]

cols_of_interest_cir = [
    'starttimeperiod', 
    'endtimeperiod', 
    'aep_endtime_utc', 
    'timezoneoffset',
    'aep_derived_uom', 
    'aep_srvc_qlty_idntfr', 
    'aep_usage_dt', 
    'value_sum',
    'counts', 
    'value_mean'
]

In [ ]:
df_outage_dir = os.path.join(Utilities.get_local_data_dir(), r'sample_outages\outg_rec_nb_11751094')
#df_outage_dir = os.path.join(Utilities.get_local_data_dir(), r'sample_outages_full\outg_rec_nb_11751094')
df_circuit_dir = os.path.join(Utilities.get_local_data_dir(), r'sample_circuits\outg_rec_nb_11751094')
#df_circuit_dir = os.path.join(Utilities.get_local_data_dir(), r'sample_circuits\Batch2_LessCounts\outg_rec_nb_11751094')

out_t_beg_loc = pd.to_datetime('2020-06-21 10:52:00')
out_t_end_loc = pd.to_datetime('2020-06-21 12:50:00')
no_outage_center_loc = pd.to_datetime('2020-08-15')

In [ ]:
#TODO WEIRD stuff going on for beginning few months of voltage!
# df_outage_dir = os.path.join(Utilities.get_local_data_dir(), r'sample_outages\outg_rec_nb_11770899')
# out_t_beg_loc = pd.to_datetime('2020-07-01 14:18:00')
# out_t_end_loc = pd.to_datetime('2020-07-01 18:50:00')
# no_outage_center_loc = pd.to_datetime('2021-04-01')

In [ ]:
freqs=['H', '4H', 'D', 'MS']
time_col_for_agg='endtimeperiod_utc'
other_grouper_cols=['serialnumber']
build_agg_dfs=True
other_cols_to_keep = ['prem_nb', 'srvc_pole_nb', 'trsf_pole_nb', 'station_nb', 
                      'xfmr_nb', 'aep_derived_uom', 'aep_srvc_qlty_idntfr']
other_cols_to_keep_agg = ['aep_derived_uom', 'aep_srvc_qlty_idntfr']
agg_cols=['value', 'annual_kwh']
agg_types=['mean']
mix_agg_functions=False

# Should not be any overlap between other_cols_to_keep and agg_cols
assert(len(set(other_cols_to_keep).intersection(set(agg_cols)))==0)
assert(len(set(other_cols_to_keep_agg).intersection(set(agg_cols)))==0)

In [ ]:
df_key = 'df'
df_agg_key = 'df_agg'

# Load Outage data

In [ ]:
kwh_vlt_dfs_dict = AMINonVee.assemble_kwh_vlt_dfs_from_saved_csvs(file_dir=df_outage_dir, glob_pattern=r'outg_rec_nb_*.csv', 
                                                        cols_of_interest=cols_of_interest, 
                                                        verbose=True)
df_kwh_15T = kwh_vlt_dfs_dict['kwh']
df_vlt_15T = kwh_vlt_dfs_dict['vlt']
# #-------------------------------------------------------------------------------------------------
dfs_kwh_dict = AMINonVee.build_time_resampled_dfs(df_kwh_15T, base_freq='15T', freqs=freqs, 
                                 other_grouper_cols=other_grouper_cols, other_cols_to_keep=other_cols_to_keep, 
                                 build_agg_dfs=build_agg_dfs, time_col_for_agg=time_col_for_agg, 
                                 agg_cols=agg_cols, agg_types=agg_types, 
                                 other_cols_to_keep_agg=other_cols_to_keep_agg, mix_agg_functions=mix_agg_functions, 
                                 df_key=df_key, df_agg_key=df_agg_key)
#-----
assert(df_kwh_15T.equals(dfs_kwh_dict['15T'][df_key]))
#-------------------------------------------------------------------------------------------------
dfs_vlt_dict = AMINonVee.build_time_resampled_dfs(df_vlt_15T, base_freq='15T', freqs=freqs, 
                                 other_grouper_cols=other_grouper_cols, other_cols_to_keep=other_cols_to_keep, 
                                 build_agg_dfs=build_agg_dfs, time_col_for_agg=time_col_for_agg, 
                                 agg_cols=agg_cols, agg_types=agg_types, 
                                 other_cols_to_keep_agg=other_cols_to_keep_agg, mix_agg_functions=mix_agg_functions, 
                                 df_key=df_key, df_agg_key=df_agg_key)
#-----
assert(df_vlt_15T.equals(dfs_vlt_dict['15T'][df_key]))
#-------------------------------------------------------------------------------------------------

In [ ]:
# dfs_kwh_dict['H']['df_agg']

# Load Circuit data

In [ ]:
#TODO!!!!!!!!!!!!!!!!!!
#TODO!!!!!!!!!!!!!!!!!!
#TODO!!!!!!!!!!!!!!!!!!
# There seems to be inconsistency in the database
# For some, each has a 'RECEIVED' and 'DELIVERED'
# For others, each has a 'RECEIVED' and 'TOTAL'
# I have not found a case yet with all three ('RECEIVED', 'DELIVERED', and 'TOTAL')
# It seems that the 'RECEIVED' counts is typically correct
# However, may want to simply take the max of n_TOTAL+n_DELIVERED and n_RECEIVED

kwh_vlt_dfs_cir_dict = AMINonVee.assemble_kwh_vlt_dfs_from_saved_csvs(file_dir=df_circuit_dir, glob_pattern=r'outg_rec_nb_*q[0-9].csv', 
                                                            value_cols=['value_sum', 'counts', 'value_mean'], 
                                                            cols_of_interest=cols_of_interest_cir, 
                                                            combine_kwh_delivered_and_received=False, 
                                                            merge_and_groupby_cols=['aep_endtime_utc'], 
                                                            verbose=True)
df_cir_kwh_15T = kwh_vlt_dfs_cir_dict['kwh']
df_cir_vlt_15T = kwh_vlt_dfs_cir_dict['vlt']

##TODO Currently, counts includes Null values as well.  The code below changes it back
df_cir_kwh_15T = df_cir_kwh_15T[df_cir_kwh_15T['value_sum'].notna()]
df_cir_kwh_15T.loc[df_cir_kwh_15T['value_sum']!=0,'counts']=round(df_cir_kwh_15T[df_cir_kwh_15T['value_sum']!=0]['value_sum']/df_cir_kwh_15T[df_cir_kwh_15T['value_sum']!=0]['value_mean'])
df_cir_kwh_15T['counts'] = df_cir_kwh_15T['counts'].astype(int)

df_cir_vlt_15T = df_cir_vlt_15T[df_cir_vlt_15T['value_sum'].notna()]
df_cir_vlt_15T.loc[df_cir_vlt_15T['value_sum']!=0,'counts']=round(df_cir_vlt_15T[df_cir_vlt_15T['value_sum']!=0]['value_sum']/df_cir_vlt_15T[df_cir_vlt_15T['value_sum']!=0]['value_mean'])
df_cir_vlt_15T['counts'] = df_cir_vlt_15T['counts'].astype(int)

#-------------------------------------------------------------------------------------------------
rename_agg_cols_cir={'value_sum':'value_sum_cir', 'counts':'counts_cir', 'value_mean':'value_mean_cir'}
df_cir_kwh_15T = df_cir_kwh_15T.rename(columns=rename_agg_cols_cir)
df_cir_vlt_15T = df_cir_vlt_15T.rename(columns=rename_agg_cols_cir)
agg_cols_cir=list(rename_agg_cols_cir.values())

#TODO!!!!!!!!!!!!!!!!!!
#TODO!!!!!!!!!!!!!!!!!!
#TODO!!!!!!!!!!!!!!!!!!
# Need to handle how delivered, received, total will be combined before re-indexing!
# For now, let's simply use 'TOTAL' from kwh
df_cir_kwh_15T_FULL = df_cir_kwh_15T.copy()
#df_cir_kwh_15T = df_cir_kwh_15T[df_cir_kwh_15T['aep_srvc_qlty_idntfr']=='TOTAL'].copy()

# TODO!!!!!!!!!!!!!!!!!!!!!
# How to handle different aep_srvc_qlty_idntfr for vlt?
# ['INSTVA1', 'INSTVC1', 'INSTVB1', 'AVG']
# For now, 'AVG' from vlt
df_cir_vlt_15T_FULL = df_cir_vlt_15T.copy()
df_cir_vlt_15T = df_cir_vlt_15T[df_cir_vlt_15T['aep_srvc_qlty_idntfr']=='AVG'].copy()

## Trying to combine total, received, and delivered for circuit data

In [ ]:
def build_circuit_non_vals(df, non_val_cols, 
                           aep_srvc_qlty_idntfr_col='aep_srvc_qlty_idntfr', 
                           aep_srvc_qlty_idntfr_vals={'rec':'RECEIVED', 'del':'DELIVERED', 'tot':'TOTAL'}):
    # Typically, df_rec, df_del, and df_tot will not all contain to entire set of
    # dates.  To simplify the merge, first grab the total set of non_vals_df.
    # This is achieved using the combine_first method, which will keep the first non-null
    # element between the two datasets.
    # Then, merges can be done using just the value_cols and time_idx.
    # The non_vals_df will be combined with the vals_df in the end
    #------------------------
    df_rec = df[df[aep_srvc_qlty_idntfr_col]==aep_srvc_qlty_idntfr_vals['rec']]
    df_del = df[df[aep_srvc_qlty_idntfr_col]==aep_srvc_qlty_idntfr_vals['del']]
    df_tot = df[df[aep_srvc_qlty_idntfr_col]==aep_srvc_qlty_idntfr_vals['tot']]
    #------------------------
    non_vals_df = df_del[non_val_cols].combine_first(df_rec[non_val_cols])
    non_vals_df = non_vals_df.combine_first(df_tot[non_val_cols])
    assert(non_vals_df.isna().sum().sum()==0) #TODO should this assert remain?
    return non_vals_df

In [ ]:
def get_circuit_total_vals_for_time_idx(gp1):
    # gp1 should be a DataFrame from a grouped call on time index
    assert(gp1.index.nunique()==1)
    
    # Should only be 'RECEIVED', 'DELIVERED', and 'TOTAL' as possible
    # values for aep_srvc_qlty_idntfr below
    aep_srvc_qlty_idntfrs = gp1['aep_srvc_qlty_idntfr'].tolist()
    assert(len(aep_srvc_qlty_idntfrs)<=3)
    
    if 'RECEIVED' in aep_srvc_qlty_idntfrs:
        received = gp1[gp1['aep_srvc_qlty_idntfr']=='RECEIVED']
        assert(received.shape[0]==1)
        received = received.iloc[0][['value_sum_cir', 'counts_cir']].to_dict()
    else:
        received = {'value_sum_cir':0, 'counts_cir':0}
    #-----
    if 'DELIVERED' in aep_srvc_qlty_idntfrs:
        delivered = gp1[gp1['aep_srvc_qlty_idntfr']=='DELIVERED']
        assert(delivered.shape[0]==1)
        delivered = delivered.iloc[0][['value_sum_cir', 'counts_cir']].to_dict()
    else:
        delivered = {'value_sum_cir':0, 'counts_cir':0}
    #-----
    if 'TOTAL' in aep_srvc_qlty_idntfrs:
        total = gp1[gp1['aep_srvc_qlty_idntfr']=='TOTAL']
        assert(total.shape[0]==1)
        total = total.iloc[0][['value_sum_cir', 'counts_cir']].to_dict()
    else:
        total = {'value_sum_cir':0, 'counts_cir':0}
    #----------------------------------------------
    net_sum = total['value_sum_cir']+delivered['value_sum_cir']-received['value_sum_cir']
    net_counts = max(received['counts_cir'], total['counts_cir']+delivered['counts_cir'])
    net_mean = net_sum/net_counts
    
    return {'value_sum_cir':net_sum, 
            'counts_cir':net_counts, 
            'value_mean_cir':net_mean}

In [ ]:
def get_ciruit_total_vals_method1(df, non_val_cols, 
                                  aep_srvc_qlty_idntfr_col='aep_srvc_qlty_idntfr', 
                                  aep_srvc_qlty_idntfr_vals={'rec':'RECEIVED', 'del':'DELIVERED', 'tot':'TOTAL'}):
    results_series = df.groupby(level=0).apply(get_circuit_total_vals_for_time_idx)
    results_df = pd.DataFrame.from_dict(results_series.to_dict(), orient='index')

    non_vals_df = build_circuit_non_vals(df, non_val_cols, 
                                         aep_srvc_qlty_idntfr_col=aep_srvc_qlty_idntfr_col, 
                                         aep_srvc_qlty_idntfr_vals=aep_srvc_qlty_idntfr_vals)
    assert(non_vals_df.shape[0]==results_df.shape[0])
    results_df = results_df.merge(non_vals_df, how='inner', left_index=True, right_index=True)
    results_df[aep_srvc_qlty_idntfr_col] = 'TOTAL'
    return results_df

In [ ]:
def get_ciruit_total_vals_method2(df, non_val_cols, 
                                  value_cols_dict = {'value_sum_cir_col':'value_sum_cir', 
                                                     'counts_cir_col':'counts_cir', 
                                                     'value_mean_cir_col':'value_mean_cir'}, 
                                  remove_intermediate_cols=True, 
                                  aep_srvc_qlty_idntfr_col='aep_srvc_qlty_idntfr', 
                                  aep_srvc_qlty_idntfr_vals={'rec':'RECEIVED', 'del':'DELIVERED', 'tot':'TOTAL'}, 
                                  maintain_original_col_order=True):
    # TODO How to enforce assert(len(aep_srvc_qlty_idntfrs)<=3)
    #---------------------------------------------------
    expected_keys = ['counts_cir_col', 'value_mean_cir_col', 'value_sum_cir_col']
    assert(len(set(value_cols_dict.keys()).intersection(set(expected_keys)))==3)    
    value_cols = list(value_cols_dict.values())
    #---------------------------------------------------
    df_rec = df[df[aep_srvc_qlty_idntfr_col]==aep_srvc_qlty_idntfr_vals['rec']]
    df_del = df[df[aep_srvc_qlty_idntfr_col]==aep_srvc_qlty_idntfr_vals['del']]
    df_tot = df[df[aep_srvc_qlty_idntfr_col]==aep_srvc_qlty_idntfr_vals['tot']]
    #---------------------------------------------------    
    # This essentially enforces assert(len(aep_srvc_qlty_idntfrs)<=3)
    assert(df_rec.shape[0]==df_rec.index.nunique())
    assert(df_del.shape[0]==df_del.index.nunique())
    assert(df_tot.shape[0]==df_tot.index.nunique())
    #---------------------------------------------------
    return_df = df_del[value_cols].merge(df_rec[value_cols], 
                                         left_index=True, right_index=True, 
                                         how='outer', suffixes=('_del', '_rec')) 
    # Note: Suffix argument will do nothing here, as df_tot contains value_cols, whereas
    #       return_df currently contains [f'x_{del}' for x in value_cols] and [f'x_{rec}' for x in value_cols]
    #       Therefore, there are no overlapping column names.  As such, we must rename using .rename
    return_df = return_df.merge(df_tot[value_cols], left_index=True, right_index=True, how='outer')
    return_df = return_df.rename(columns = {x:f'{x}_tot' for x in value_cols})
    # I used an outer merge so there will always been an entry for delivered, received, and total in return_df
    # However, if one of these did not exist in the original, the outer merge will leave a NaN value in the cell
    # Therefore, I call return_df.fillna(0) to replace any of these NaNs with 0
    return_df = return_df.fillna(0)
    #---------------------------------------------------
    value_sum_cir_col = value_cols_dict['value_sum_cir_col']
    counts_cir_col = value_cols_dict['counts_cir_col']
    value_mean_cir_col = value_cols_dict['value_mean_cir_col']
    #---------------------------------------------------
    # Made a list of newly created columns, which will be need if remove_intermediate_cols==True
    new_col_tags = ['_del', '_rec', '_tot']
    new_cols = [value_col+tag for value_col in value_cols for tag in new_col_tags]
    new_cols.append(f"{counts_cir_col}_del_plus_tot")
    #---------------------------------------------------
    return_df[f"{counts_cir_col}_del_plus_tot"] = (return_df[f"{counts_cir_col}_del"] +
                                                   return_df[f"{counts_cir_col}_tot"])
    return_df[counts_cir_col] = return_df[[f"{counts_cir_col}_rec", 
                                           f"{counts_cir_col}_del_plus_tot"]].max(axis=1)
    return_df[value_sum_cir_col] = (return_df[f"{value_sum_cir_col}_tot"] +
                                    return_df[f"{value_sum_cir_col}_del"] -
                                    return_df[f"{value_sum_cir_col}_rec"])
    return_df[value_mean_cir_col] = return_df[value_sum_cir_col]/return_df[value_cols_dict['counts_cir_col']]
    return_df[counts_cir_col]=return_df[counts_cir_col].astype('int64')
    #---------------------------------------------------
    if remove_intermediate_cols:
        return_df = return_df.drop(columns=new_cols)
    #---------------------------------------------------
    non_vals_df = build_circuit_non_vals(df, non_val_cols, 
                                         aep_srvc_qlty_idntfr_col=aep_srvc_qlty_idntfr_col, 
                                         aep_srvc_qlty_idntfr_vals=aep_srvc_qlty_idntfr_vals)
    assert(non_vals_df.shape[0]==return_df.shape[0])
    return_df = return_df.merge(non_vals_df, how='inner', left_index=True, right_index=True)
    return_df[aep_srvc_qlty_idntfr_col] = 'TOTAL'
    if maintain_original_col_order:
        cols_from_og = [x for x in df.columns if x in return_df.columns]
        return_df = Utilities_df.move_cols_to_front(return_df, cols_from_og)
    return return_df

In [ ]:
non_val_cols = ['starttimeperiod', 'endtimeperiod', 'aep_endtime_utc', 'timezoneoffset',
                'aep_derived_uom', 'aep_usage_dt', 'aep_endtime_utc_from_timestamp', 
                'starttimeperiod_utc', 'endtimeperiod_utc']
value_cols_dict = {'value_sum_cir_col':'value_sum_cir', 
                   'counts_cir_col':'counts_cir', 
                   'value_mean_cir_col':'value_mean_cir'}

In [ ]:
# dfs_cir_kwh_dict['H']['df']

### Test small sample of data

In [ ]:
test_df = df_cir_kwh_15T_FULL.loc[:'2018-05-28 21:30:00'].copy() # First ~1000 entries
#---------------------------------------------
n_rep = 10

start = time.time()
for i in range(n_rep):
    results_method1 = get_ciruit_total_vals_method1(test_df, non_val_cols)
time_1 = time.time() - start

start = time.time()
for i in range(n_rep):
    results_method2 = get_ciruit_total_vals_method2(test_df, non_val_cols, 
                                                    value_cols_dict=value_cols_dict, remove_intermediate_cols=True)
time_2 = time.time() - start

print(f'n_rep = {n_rep}')
print(f'Avg time_1 = {time_1/n_rep}')
print(f'Avg time_2 = {time_2/n_rep}')

assert(len(results_method1.columns)==len(results_method2.columns))
assert(len(set(results_method2.columns).intersection(set(results_method1.columns)))==len(results_method2.columns))
# Made column ordering same
print(f'results_method2==results_method1: {results_method2[results_method1.columns].equals(results_method1)}')
# results_method2[results_method1.columns].equals(results_method1)

### Test full dataset

In [ ]:
test_df_full = df_cir_kwh_15T_FULL.copy()
#-------------------------------------------------------
# This takes some time to run, maybe 10 minutes?  I can't remember
# Certainly much longer than method 2
are_you_sure = False
if are_you_sure:
    start = time.time()
    results_method1_full = get_ciruit_total_vals_method1(test_df_full, non_val_cols)
    time_1 = time.time() - start
    print(f'time_1 = {time_1}')
#-------------------------------------------------------
start = time.time()
results_method2_full = get_ciruit_total_vals_method2(test_df_full, non_val_cols, 
                                                     value_cols_dict=value_cols_dict, remove_intermediate_cols=True)
time_2 = time.time() - start
print(f'time_2 = {time_2}')
#-------------------------------------------------------
if are_you_sure:
    assert(len(results_method1_full.columns)==len(results_method2_full.columns))
    assert(len(set(results_method2_full.columns).intersection(set(results_method1_full.columns)))==len(results_method2_full.columns))
    # Made column ordering same
    results_method2_full[results_method1_full.columns].equals(results_method1_full)

### Use combine method on full dataset and then resample/aggregate

In [ ]:
non_val_cols = ['starttimeperiod', 'endtimeperiod', 'aep_endtime_utc', 'timezoneoffset',
                'aep_derived_uom', 'aep_usage_dt', 'aep_endtime_utc_from_timestamp', 
                'starttimeperiod_utc', 'endtimeperiod_utc']
value_cols_dict = {'value_sum_cir_col':'value_sum_cir', 
                   'counts_cir_col':'counts_cir', 
                   'value_mean_cir_col':'value_mean_cir'}

In [ ]:
dfs_kwh_dict['H']['df']

In [ ]:
df_cir_kwh_15T = get_ciruit_total_vals_method2(df_cir_kwh_15T, non_val_cols, 
                                               value_cols_dict=value_cols_dict, remove_intermediate_cols=True)
#-------------------------------------------------------------------------------------------------
# TODO!!!!!!!!!!!!!!!!!
# Also, would expect e.g. dfs_cir_kwh_dict['H'][df_key]==dfs_cir_kwh_dict['H'][df_agg_key]
# However, this is close but not exactly true
# It appears there are some times with duplicate entries
# Compare the size of df_cir_kwh_15T[df_cir_kwh_15T['aep_srvc_qlty_idntfr']=='TOTAL']
# to the number of unique indices
#
#

dfs_cir_kwh_dict = AMINonVee.build_time_resampled_dfs(df_cir_kwh_15T, freqs=freqs, other_grouper_cols=[], 
                                     build_agg_dfs=build_agg_dfs, time_col_for_agg=time_col_for_agg, 
                                     agg_cols=agg_cols_cir, agg_types=['mean', 'sum'], 
                                     df_key=df_key, df_agg_key=df_agg_key)
#-----
assert(df_cir_kwh_15T.equals(dfs_cir_kwh_dict['15T'][df_key]))
#-------------------------------------------------------------------------------------------------
dfs_cir_vlt_dict = AMINonVee.build_time_resampled_dfs(df_cir_vlt_15T, freqs=freqs, other_grouper_cols=[], 
                                     build_agg_dfs=build_agg_dfs, time_col_for_agg=time_col_for_agg, agg_cols=agg_cols_cir, 
                                     df_key=df_key, df_agg_key=df_agg_key)
#-----
assert(df_cir_vlt_15T.equals(dfs_cir_vlt_dict['15T'][df_key]))
#-------------------------------------------------------------------------------------------------

In [ ]:
print('Checking endtimeperiod_utc against aep_endtime_utc_from_timestamp')
print('df_kwh_15T:     ', all(df_kwh_15T['endtimeperiod_utc']==df_kwh_15T['aep_endtime_utc_from_timestamp']))
print('df_vlt_15T:     ', all(df_vlt_15T['endtimeperiod_utc']==df_vlt_15T['aep_endtime_utc_from_timestamp']))
#-----
print('df_cir_kwh_15T: ', all(df_cir_kwh_15T['endtimeperiod_utc']==df_cir_kwh_15T['aep_endtime_utc_from_timestamp']))
print('df_cir_vlt_15T: ', all(df_cir_vlt_15T['endtimeperiod_utc']==df_cir_vlt_15T['aep_endtime_utc_from_timestamp']))

In [ ]:
# df_kwh_15T.head()

In [ ]:
tmpppp = dfs_cir_kwh_dict['H']['df'].copy()
all(tmpppp['mean_TRS value_sum_cir']/tmpppp['mean_TRS counts_cir']==tmpppp['mean_TRS value_mean_cir'])

In [ ]:
tmpppp[tmpppp['mean_TRS value_sum_cir']/tmpppp['mean_TRS counts_cir']!=tmpppp['mean_TRS value_mean_cir']]

# Convert out_t_beg_loc and out_t_end_loc to UTC

In [ ]:
unique_tz_offsets = pd.to_timedelta(df_kwh_15T['timezoneoffset'].unique()).tolist()
# Should be complete hour
assert(all(x.total_seconds()%3600==0 for x in unique_tz_offsets))
unique_tz_offsets = [round(x.total_seconds()/3600) for x in unique_tz_offsets]
found_tz = Utilities_dt.determine_us_timezone(unique_tz_offsets, assert_found=True)
#--------------------
out_t_beg = pd.to_datetime(out_t_beg_loc).tz_localize(found_tz).tz_convert(None)
out_t_end = pd.to_datetime(out_t_end_loc).tz_localize(found_tz).tz_convert(None)
no_outage_center = pd.to_datetime(no_outage_center_loc).tz_localize(found_tz).tz_convert(None)

In [ ]:
for idx, gp in df_cir_kwh_15T_FULL.groupby(level=0):
    value_counts = gp['aep_srvc_qlty_idntfr'].value_counts()
    if len(value_counts)<2:
        print(idx)

In [ ]:
for idx, gp in df_kwh_15T[df_kwh_15T['serialnumber']==880693914].groupby(level=0):
    if len(gp)>1:
        print(idx)

In [ ]:
df_cir_kwh_15T_FULL.groupby('starttimeperiod').get_group('2018-05-25T07:15:00-04:00')

In [ ]:
run=False
if run:
    total=0
    #for idx,grp in df_cir_kwh_15T_FULL.groupby('starttimeperiod'):
    for idx,grp in df_cir_kwh_15T_FULL.groupby(level=0):
        if grp.shape[0]!=2 and grp.shape[0]!=3:
            print(idx, grp.shape[0], '\n')
            #print(grp.shape[0])
        total += grp.shape[0]
    print(total)

In [ ]:
missing = find_missing_time_idxs(df_kwh_15T, '15T')

In [ ]:
missing

In [ ]:
test_df

In [ ]:
test_df.groupby(['time_idx'])

In [ ]:
df_cir_kwh_15T_FULL[df_cir_kwh_15T_FULL['starttimeperiod'].isin(['2019-11-03T01:30:00-05:00', 
                                                                 '2019-11-03T01:45:00-05:00', 
                                                                 '2019-11-03T02:00:00-05:00'])]

In [ ]:
df_cir_kwh_15T_FULL[df_cir_kwh_15T_FULL['starttimeperiod']=='2020-11-01T01:45:00-05:00']

In [ ]:
df_cir_kwh_15T_FULL['endtimeperiod_utc']

### Figure out DST

In [ ]:
cols_to_move = ['starttimeperiod', 'endtimeperiod', 
                'starttimeperiod_utc', 'endtimeperiod_utc', 
                'aep_endtime_utc_from_timestamp', 'aep_endtime_utc']
df_kwh_15T = Utilities_df.move_cols_to_front(df_kwh_15T, cols_to_move)

In [ ]:
tmp_df = df_kwh_15T[df_kwh_15T['serialnumber']==880687439].copy()

In [ ]:
tmp_df[tmp_df['endtimeperiod_utc']!=tmp_df['aep_endtime_utc_from_timestamp']]

In [ ]:
df_kwh_15T[df_kwh_15T['endtimeperiod_utc']!=df_kwh_15T['aep_endtime_utc_from_timestamp']]

In [ ]:
tmp_df[tmp_df['starttimeperiod_utc']=='2020-03-08 01:45:00']

In [ ]:
tmp_df[tmp_df['starttimeperiod_utc']=='2018-11-04 01:45:00']

In [ ]:
tmp_df[(tmp_df['starttimeperiod_utc']>='2018-11-04 00:45:00') & (tmp_df['starttimeperiod_utc']<='2018-11-04 02:15:00')]

In [ ]:
tmp_df[tmp_df['starttimeperiod_utc']>tmp_df['endtimeperiod_utc']]

In [ ]:
tmp_df[(tmp_df['starttimeperiod_utc']>='2019-11-03 00:30:00') & (tmp_df['starttimeperiod_utc']<='2019-11-03 02:15:00')]

In [ ]:
# Need to do the same thing for endtimeperiod_utc and starttimeperiod_utc
# and probably for endtimeperiod and starttimeperiod
wrong_start_end_df = tmp_df[tmp_df['endtimeperiod_utc']-tmp_df['starttimeperiod_utc']!=pd.to_timedelta('15T')]

In [ ]:
wrong_start_end_df

In [ ]:
wrong_start_end_df.iloc[0]

In [ ]:
pd.to_datetime('2019-11-03T01:45:00-04:00').tz_convert(None)

In [ ]:
pd.to_datetime('2019-11-03T00:45:00-04:00').tz_convert(None)

In [ ]:
pd.to_datetime('2019-11-03T01:45:00-03:00').tz_convert(None)

In [ ]:
def try_to_find_correct_starttimeperiod(starttimeperiod, endtimeperiod, freq='15T'):
    # end-start = observed
    # ==> end - start - observed + desired = desired
    # ==> end - (start + (observed-desired)) = desired
    # ==> start --> start + (observed-desired)
    starttimeperiod_utc = pd.to_datetime(starttimeperiod).tz_convert(None)
    endtimeperiod_utc = pd.to_datetime(endtimeperiod).tz_convert(None)
    time_delta_observed = endtimeperiod_utc-starttimeperiod_utc
    time_delta_desired = pd.to_timedelta(freq)
    start_time_change_needed = time_delta_observed-time_delta_desired
    #--------------------------------------------
    starttimeperiod_strp = datetime.datetime.strptime(starttimeperiod, '%Y-%m-%dT%H:%M:%S%z')
    endtimeperiod_strp = datetime.datetime.strptime(endtimeperiod, '%Y-%m-%dT%H:%M:%S%z')
    start_tz_offset_in_sec = starttimeperiod_strp.tzinfo.utcoffset(datetime.datetime.utcnow()).total_seconds()
    end_tz_offset_in_sec = endtimeperiod_strp.tzinfo.utcoffset(datetime.datetime.utcnow()).total_seconds()

    # There do exist timezone shifts of 30min, or ever 45 min I believe
    # But I shouldn't see any such tzs in my data
    assert(start_tz_offset_in_sec%3600==0)
    assert(end_tz_offset_in_sec%3600==0)

    start_tz_offset_hours = int(start_tz_offset_in_sec/3600)
    end_tz_offset_hours = int(end_tz_offset_in_sec/3600)
    #--------------------------------------------
    # In the United States:
    #   Spring DST occurs in March
    #   Fall DST occurs in November
    is_spring = endtimeperiod_utc.date().month==3
    is_fall = endtimeperiod_utc.date().month==11
    assert(is_spring+is_fall==1)    

    # Can either subtract hour from time, add hour to tz shift, or both I suppose
    # However, there is a restriction:
    #     The only time the timezone of starttimeperiod differs from endtimeperiod is at the DST transition
    #       In spring, it is allowed for starttimeperiod.tz < endtimeperiod.tz by 1 (i.e. endtimeperiod.tz-starttimeperiod.tz==1)
    #       In fall, it is allowed for starttimeperiod.tz > endtimeperiod.tz by 1 (i.e. starttimeperiod.tz-endtimeperiod.tz==1)
    starttimeperiod_utc_new = starttimeperiod_utc+start_time_change_needed
    start_tz_offset_new = pd.to_timedelta(start_tz_offset_hours, 'H')-start_time_change_needed

    is_shifted_tz_allowed = False
    if is_fall and start_tz_offset_new-pd.to_timedelta(end_tz_offset_hours, 'H')==pd.to_timedelta('1H'):
        is_shifted_tz_allowed=True
    if is_spring and pd.to_timedelta(end_tz_offset_hours, 'H')-start_tz_offset_new==pd.to_timedelta('1H'):
        is_shifted_tz_allowed=True

    outcomes = []
    psb_1_time = starttimeperiod_utc_new
    psb_1_shift = pd.to_timedelta(start_tz_offset_hours, 'H')
    psb_1_full = psb_1_time.tz_localize('UTC').tz_convert(datetime.timezone(psb_1_shift))
    result_1 = pd.to_datetime(endtimeperiod).tz_localize(None)-psb_1_full.tz_localize(None)==time_delta_desired
    outcomes.append((result_1, psb_1_full))

    if is_shifted_tz_allowed:
        psb_2_time = starttimeperiod_utc
        psb_2_shift = start_tz_offset_new
        psb_2_full = psb_2_time.tz_localize('UTC').tz_convert(datetime.timezone(psb_2_shift))
        result_2 = pd.to_datetime(endtimeperiod).tz_localize(None)-psb_2_full.tz_localize(None)==time_delta_desired
        outcomes.append((result_2, psb_2_full))


        psb_3_time = starttimeperiod_utc_new
        psb_3_shift = start_tz_offset_new
        psb_3_full = psb_3_time.tz_localize('UTC').tz_convert(datetime.timezone(psb_3_shift))
        result_3 = pd.to_datetime(endtimeperiod).tz_localize(None)-psb_3_full.tz_localize(None)==time_delta_desired
        outcomes.append((result_3, psb_3_full))


    suitable = [x for x in outcomes if x[0]]
    # There should really only be one correct combination
    assert(len(suitable)==1)
    suitable=suitable[0][1]

    # Unfortunately psb_1_full.strftime('%Y-%m-%dT%H:%M:%S%z') would give e.g. '2019-11-03T00:45:00-0400'
    #   And there doesn't appear to be an option to output tz as -04:00 instead of -0400
    # print(psb_1_full) gives, e.g. '2019-11-03 00:45:00-04:00'
    #   The substitution below adds the T between date and time
    starttimeperiod_correct = re.sub(r'(\d{4}-\d{2}-\d{2}) (\d{2}:\d{2}:\d{2})([+-]\d{2}:\d{2})', r'\1T\2\3', str(suitable))
    return starttimeperiod_correct

In [ ]:
print(tmp_df.loc['2019-11-03 01:45:00'][['starttimeperiod', 'endtimeperiod', 'aep_endtime_utc_from_timestamp']])

In [ ]:
tmp_df[tmp_df['starttimeperiod_utc']>tmp_df['endtimeperiod_utc']]

In [ ]:
tmp_df[tmp_df['starttimeperiod']=='2019-11-03T00:30:00-04:00']

In [ ]:
tmp_df.loc['2019-11-03 00:00:00':'2019-11-03 12:00:00']

In [ ]:
tmp_df.loc['2019-11-03 00:00:00':'2019-11-03 12:00:00'].sort_values(by='starttimeperiod_utc')

In [ ]:
read_df = pd.read_csv(os.path.join(Utilities.get_local_data_dir(), r'sample_outages\outg_rec_nb_11751094\outg_rec_nb_11751094_2019_q4.csv'))
read_df = read_df[read_df['serialnumber']==880687439]
read_df = read_df[(read_df['aep_derived_uom']=='KWH') & (read_df['aep_srvc_qlty_idntfr']=='TOTAL')]
read_df = read_df[['starttimeperiod', 'endtimeperiod','aep_endtime_utc', 'timezoneoffset']]
read_df = read_df.sort_values(by='aep_endtime_utc', ignore_index=True)
read_df

In [ ]:
read_df

In [ ]:
read_df[read_df['starttimeperiod']=='2019-11-03T01:00:00-04:00']

In [ ]:
read_df[read_df['starttimeperiod']=='2019-11-03T03:00:00-05:00']

In [ ]:
read_df.iloc[3164:3184]

In [ ]:
#'2021-03-14T01:45:00-04:00'

In [ ]:
read_df = pd.read_csv(os.path.join(Utilities.get_local_data_dir(), r'sample_outages\outg_rec_nb_11751094\outg_rec_nb_11751094_2021_q1.csv'))
read_df = read_df[read_df['serialnumber']==880687439]
read_df = read_df[(read_df['aep_derived_uom']=='KWH') & (read_df['aep_srvc_qlty_idntfr']=='TOTAL')]
read_df = read_df[['starttimeperiod', 'endtimeperiod','aep_endtime_utc', 'timezoneoffset']]
read_df = read_df.sort_values(by='aep_endtime_utc', ignore_index=True)
read_df = convert_timestamp_to_utc_in_df(read_df, 'aep_endtime_utc')

In [ ]:
read_df[read_df['starttimeperiod']=='2021-03-14T01:45:00-04:00']

In [ ]:
read_df.iloc[6900:6920]

In [ ]:
starttimeperiod = '2021-03-14T01:45:00-04:00' # Should really be '2021-03-14T02:45:00-04:00'?
endtimeperiod = '2021-03-14T03:00:00-04:00'

try_to_find_correct_starttimeperiod(starttimeperiod, endtimeperiod, freq='15T')

In [ ]:
tmp_df.loc['2019-11-03 04:00:00':'2019-11-03 12:00:00']

In [ ]:
tmp_df[tmp_df['starttimeperiod_utc']>tmp_df['endtimeperiod_utc']]

In [ ]:
tmp_df.loc['2019-11-03 05:15:00':'2019-11-03 12:00:00'].sort_values(by='starttimeperiod_utc')

In [ ]:
tmp_df_2=tmp_df.set_index('aep_endtime_utc_from_timestamp', drop=False).sort_index().copy()
tmp_df_2.index.name = 'time_idx'

In [ ]:
tmp_df_2['diff'] = tmp_df_2['aep_endtime_utc_from_timestamp'].diff()

In [ ]:
all(tmp_df_2.iloc[1:]['diff']==pd.to_timedelta('15T'))

In [ ]:
no_bueno = [(i,x) for i,x in enumerate(tmp_df_2['diff']) if x!=pd.to_timedelta('15T')]

In [ ]:
no_bueno

In [ ]:
tmp_df_2.iloc[53630:53632]

### Figure out what to do with SQL GROUP BY by working with df_kwh_15T

In [ ]:
find_non_quarter_hour_entries(df_cir_kwh_15T_FULL.index)

In [ ]:
df_cir_kwh_15T_FULL.iloc[40374]

In [ ]:
df_cir_kwh_15T_FULL[df_cir_kwh_15T_FULL['starttimeperiod_utc']=='2018-11-04 01:45:00']

In [ ]:
#TODO !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# DAYLIGHT SAVINGS!
# (apparently not) Solved: change to using utc time, which isn't affected by dst
df_cir_kwh_15T_FULL[df_cir_kwh_15T_FULL['starttimeperiod_utc']>df_cir_kwh_15T_FULL['endtimeperiod_utc']]

In [ ]:
df_cir_kwh_15T_FULL['value_mean_cir'].plot(figsize=(20,10))

In [ ]:
df_cir_kwh_15T_FULL['value_sum_cir'].plot(figsize=(20,10))

In [ ]:
# df_cir_kwh_15T_FULL

In [ ]:
expand_time = datetime.timedelta(days=3)
ax=df_cir_kwh_15T_FULL[out_t_beg-expand_time:out_t_end+expand_time]['value_mean_cir'].plot(figsize=(20,10))
ax.axvline(out_t_beg, color='red')
ax.axvline(out_t_end, color='lawngreen')

In [ ]:
expand_time = datetime.timedelta(days=3)
ax=df_cir_kwh_15T[out_t_beg-expand_time:out_t_end+expand_time]['value_mean_cir'].plot(figsize=(20,10))
ax.axvline(out_t_beg, color='red')
ax.axvline(out_t_end, color='lawngreen')

In [ ]:
expand_time = datetime.timedelta(days=3)
ax=df_cir_kwh_15T_FULL[df_cir_kwh_15T_FULL['aep_srvc_qlty_idntfr']=='TOTAL'][out_t_beg-expand_time:out_t_end+expand_time]['value_mean_cir'].plot(figsize=(20,10))
ax.axvline(out_t_beg, color='red')
ax.axvline(out_t_end, color='lawngreen')

In [ ]:
expand_time = datetime.timedelta(days=3)
ax=df_cir_kwh_15T_FULL[df_cir_kwh_15T_FULL['aep_srvc_qlty_idntfr']=='DELIVERED'][out_t_beg-expand_time:out_t_end+expand_time]['value_mean_cir'].plot(figsize=(20,10))
ax.axvline(out_t_beg, color='red')
ax.axvline(out_t_end, color='lawngreen')

In [ ]:
expand_time = datetime.timedelta(days=3)
ax=df_cir_kwh_15T_FULL[df_cir_kwh_15T_FULL['aep_srvc_qlty_idntfr']=='RECEIVED'][out_t_beg-expand_time:out_t_end+expand_time]['value_mean_cir'].plot(figsize=(20,10))
ax.axvline(out_t_beg, color='red')
ax.axvline(out_t_end, color='lawngreen')

In [ ]:
expand_time = datetime.timedelta(days=3)
ax=df_cir_kwh_15T_FULL[df_cir_kwh_15T_FULL['aep_srvc_qlty_idntfr']=='TOTAL'][out_t_beg-expand_time:out_t_end+expand_time]['value_mean_cir'].plot(figsize=(20,10), label='TOTAL', color='red')
ax=df_cir_kwh_15T_FULL[df_cir_kwh_15T_FULL['aep_srvc_qlty_idntfr']=='DELIVERED'][out_t_beg-expand_time:out_t_end+expand_time]['value_mean_cir'].plot(label='DELIVERED')
ax=df_cir_kwh_15T[out_t_beg-expand_time:out_t_end+expand_time]['value_mean_cir'].plot(label='AVG/NET', color='purple')
ax.axvline(out_t_beg, color='red')
ax.axvline(out_t_end, color='lawngreen')
ax.legend()

In [ ]:
expand_time = datetime.timedelta(days=3)
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
plt.subplots_adjust(hspace=0.3)

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_cir_kwh_15T, 
                                  x='starttimeperiod_utc', y='value_mean_cir', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'Hourly Average Consumption (kWh) [expand time = {expand_time}]', fontsize=25), 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                   seg_line_freq='D')

In [ ]:
# ax1.figure.legend(bbox_to_anchor=(1,1), loc='upper left')

In [ ]:
dfs_kwh_dict['H']['df']

# Correlations

In [ ]:
def get_correlation_coeff(df_1, col_1, df_2, col_2):    
    common_idxs_kwh = set(df_1.index).intersection(set(df_2.index))
    df_1_to_corr = df_1.loc[common_idxs_kwh]
    df_2_to_corr = df_2.loc[common_idxs_kwh]
    #-----
    corr = df_1_to_corr[col_1].corr(df_2_to_corr[col_2])
    return corr

In [ ]:
# Correlation method 1
kwh_corr = get_correlation_coeff(dfs_kwh_dict['MS'][df_agg_key], 'mean_TRS mean_mtrs value', 
                                 dfs_cir_kwh_dict['MS'][df_key], 'mean_TRS value_mean_cir')
vlt_corr = get_correlation_coeff(dfs_vlt_dict['MS'][df_agg_key], 'mean_TRS mean_mtrs value', 
                                 dfs_cir_vlt_dict['MS'][df_key], 'mean_TRS value_mean_cir')
print(f'kwh_corr = {kwh_corr}')
print(f'vlt_corr = {vlt_corr}')

In [ ]:
common_idxs_kwh = set(dfs_kwh_dict['D'][df_agg_key].index).intersection(set(dfs_cir_kwh_dict['D'][df_key].index))
df_kwh_D_mean_to_corr = dfs_kwh_dict['D'][df_agg_key].loc[common_idxs_kwh]
df_cir_kwh_D_to_corr = dfs_cir_kwh_dict['D'][df_key].loc[common_idxs_kwh]

In [ ]:
common_idxs_vlt = set(dfs_vlt_dict['D'][df_agg_key].index).intersection(set(dfs_cir_vlt_dict['D'][df_key].index))
df_vlt_D_mean_to_corr = dfs_vlt_dict['D'][df_agg_key].loc[common_idxs_vlt]
df_cir_vlt_D_to_corr = dfs_cir_vlt_dict['D'][df_key].loc[common_idxs_vlt]

In [ ]:
# Correlation method 1
kwh_corr = get_correlation_coeff(df_kwh_D_mean_to_corr, 'mean_TRS mean_mtrs value', 
                                 df_cir_kwh_D_to_corr, 'mean_TRS value_mean_cir')
vlt_corr = get_correlation_coeff(df_vlt_D_mean_to_corr, 'mean_TRS mean_mtrs value', 
                                 df_cir_vlt_D_to_corr, 'mean_TRS value_mean_cir')
print(f'kwh_corr = {kwh_corr}')
print(f'vlt_corr = {vlt_corr}')

In [ ]:
# Correlation method 2: kwh
# NOTE: For this method, inputs cannot contain NaNs!
not_nans_bool_out = df_kwh_D_mean_to_corr['mean_TRS mean_mtrs value'].notna()
not_nans_bool_cir = df_cir_kwh_D_to_corr['mean_TRS value_mean_cir'].notna()
not_nans_bool = not_nans_bool_out & not_nans_bool_cir

Y = df_kwh_D_mean_to_corr.loc[not_nans_bool]['mean_TRS mean_mtrs value']
Z = df_cir_kwh_D_to_corr.loc[not_nans_bool]['mean_TRS value_mean_cir']
stats.pearsonr(Y,Z)

In [ ]:
# Correlation method 2: vlt
# NOTE: For this method, inputs cannot contain NaNs!
not_nans_bool_out = df_vlt_D_mean_to_corr['mean_TRS mean_mtrs value'].notna()
not_nans_bool_cir = df_cir_vlt_D_to_corr['mean_TRS value_mean_cir'].notna()
not_nans_bool = not_nans_bool_out & not_nans_bool_cir

Y = df_vlt_D_mean_to_corr.loc[not_nans_bool]['mean_TRS mean_mtrs value']
Z = df_cir_vlt_D_to_corr.loc[not_nans_bool]['mean_TRS value_mean_cir']
stats.pearsonr(Y,Z)

In [ ]:
# df_kwh_15T.loc['2020-06-23 18:30:00'].mean()

In [ ]:
# df_kwh_15T_mean.loc['2020-06-23 18:30:00']

In [ ]:
df_kwh_15T

In [ ]:
dfs_kwh_dict['15T'][df_agg_key]

In [ ]:
expand_time = datetime.timedelta(days=3)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=dfs_kwh_dict['15T'][df_agg_key], 
                                  x='endtimeperiod_utc', y='mean_mtrs value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, data_label='Outage Mean', 
                                  title_args=dict(label=f'15T Average Consumption (kWh)', fontsize=25), 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                   seg_line_freq='D')

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_cir_kwh_15T, 
                                  x='endtimeperiod_utc', y='value_mean_cir', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, data_label='Circuit Mean')

kwh_corr = get_correlation_coeff(dfs_kwh_dict['15T'][df_agg_key], 'mean_mtrs value', 
                                 df_cir_kwh_15T, 'value_mean_cir')
plt.text(1.01, 1.01, f'Corr = {kwh_corr}', transform=ax0.transAxes, fontsize=15)


fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=dfs_vlt_dict['15T'][df_agg_key], 
                                  x='endtimeperiod_utc', y='mean_mtrs value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, data_label='Outage Mean', 
                                  title_args=dict(label=f'15T Average Voltage (V)', fontsize=25), 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                   seg_line_freq='D')
ax1.set_ylim([225,250])

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=df_cir_vlt_15T, 
                                  x='endtimeperiod_utc', y='value_mean_cir', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, data_label='Circuit Mean')

vlt_corr = get_correlation_coeff(dfs_vlt_dict['15T'][df_agg_key], 'mean_mtrs value', 
                                 df_cir_vlt_15T, 'value_mean_cir')
plt.text(1.01, 1.01, f'Corr = {vlt_corr}', transform=ax1.transAxes, fontsize=15)

fig_num +=1

In [ ]:
expand_time = datetime.timedelta(days=3)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=dfs_kwh_dict['H'][df_agg_key], 
                                  x='endtimeperiod_utc', y='mean_TRS mean_mtrs value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, data_label='Outage Mean', 
                                  title_args=dict(label=f'Hourly Average Consumption (kWh)', fontsize=25), 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                   seg_line_freq='D')

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=dfs_cir_kwh_dict['H'][df_key], 
                                  x='date', y='mean_TRS value_mean_cir', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, data_label='Circuit Mean')

kwh_corr = get_correlation_coeff(dfs_kwh_dict['H'][df_agg_key], 'mean_TRS mean_mtrs value', 
                                 dfs_cir_kwh_dict['H'][df_key], 'mean_TRS value_mean_cir')
plt.text(1.01, 1.01, f'Corr = {kwh_corr}', transform=ax0.transAxes, fontsize=15)

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=dfs_vlt_dict['H'][df_agg_key], 
                                  x='endtimeperiod_utc', y='mean_TRS mean_mtrs value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, data_label='Outage Mean', 
                                  title_args=dict(label=f'Hourly Average Voltage (V)', fontsize=25), 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                   seg_line_freq='D')
ax1.set_ylim([225,250])

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=dfs_cir_vlt_dict['H'][df_key], 
                                  x='date', y='mean_TRS value_mean_cir', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, data_label='Circuit Mean')

vlt_corr = get_correlation_coeff(dfs_vlt_dict['H'][df_agg_key], 'mean_TRS mean_mtrs value', 
                                 dfs_cir_vlt_dict['H'][df_key], 'mean_TRS value_mean_cir')
plt.text(1.01, 1.01, f'Corr = {vlt_corr}', transform=ax1.transAxes, fontsize=15)

fig_num +=1

In [ ]:
expand_time = datetime.timedelta(days=31)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=dfs_kwh_dict['D'][df_agg_key], 
                                  x='endtimeperiod_utc', y='mean_TRS mean_mtrs value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, data_label='Outage Mean', 
                                  title_args=dict(label=f'Daily Average Consumption (kWh)', fontsize=25), 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                   seg_line_freq='7D')

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=dfs_cir_kwh_dict['D'][df_key], 
                                  x='date', y='mean_TRS value_mean_cir', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, data_label='Circuit Mean')

kwh_corr = get_correlation_coeff(dfs_kwh_dict['D'][df_agg_key], 'mean_TRS mean_mtrs value', 
                                 dfs_cir_kwh_dict['D'][df_key], 'mean_TRS value_mean_cir')
plt.text(1.01, 1.01, f'Corr = {kwh_corr}', transform=ax0.transAxes, fontsize=15)

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=dfs_vlt_dict['D'][df_agg_key], 
                                  x='endtimeperiod_utc', y='mean_TRS mean_mtrs value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, data_label='Outage Mean', 
                                  title_args=dict(label=f'Daily Average Voltage (V)', fontsize=25), 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                   seg_line_freq='7D')
ax1.set_ylim([225,250])

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=dfs_cir_vlt_dict['D'][df_key], 
                                  x='date', y='mean_TRS value_mean_cir', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, data_label='Circuit Mean')

vlt_corr = get_correlation_coeff(dfs_vlt_dict['D'][df_agg_key], 'mean_TRS mean_mtrs value', 
                                 dfs_cir_vlt_dict['D'][df_key], 'mean_TRS value_mean_cir')
plt.text(1.01, 1.01, f'Corr = {vlt_corr}', transform=ax1.transAxes, fontsize=15)

fig_num +=1

# Divide outage by circuit

In [ ]:
tmp_kwh_df = dfs_kwh_dict['D'][df_agg_key].copy()
tmp_kwh_df['value div. by cir.'] = tmp_kwh_df['mean_TRS mean_mtrs value']/dfs_cir_kwh_dict['D'][df_key]['mean_TRS value_mean_cir']

tmp_vlt_df = dfs_vlt_dict['D'][df_agg_key].copy()
tmp_vlt_df['value div. by cir.'] = tmp_vlt_df['mean_TRS mean_mtrs value']/dfs_cir_vlt_dict['D'][df_key]['mean_TRS value_mean_cir']

In [ ]:
expand_time = datetime.timedelta(days=31)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=tmp_kwh_df, 
                                  x='endtimeperiod_utc', y='value div. by cir.', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, data_label='Outage Mean', 
                                  title_args=dict(label=f'Daily Average Consumption (kWh)', fontsize=25), 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                   seg_line_freq='7D')

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=tmp_vlt_df, 
                                  x='endtimeperiod_utc', y='value div. by cir.', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, data_label='Outage Mean', 
                                  title_args=dict(label=f'Daily Average Voltage (V)', fontsize=25), 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                   seg_line_freq='7D')

fig_num +=1

In [ ]:
expand_time = datetime.timedelta(days=12*31)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=dfs_kwh_dict['MS'][df_agg_key], 
                                  x='endtimeperiod_utc', y='mean_TRS mean_mtrs value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'Monthly Average Consumption (kWh)', fontsize=25), 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                   seg_line_freq='31D')

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=dfs_cir_kwh_dict['MS'][df_key], 
                                  x='date', y='mean_TRS value_mean_cir', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time)

kwh_corr = get_correlation_coeff(dfs_kwh_dict['MS'][df_agg_key], 'mean_TRS mean_mtrs value', 
                                 dfs_cir_kwh_dict['MS'][df_key], 'mean_TRS value_mean_cir')
plt.text(1.01, 1.01, f'Corr = {kwh_corr}', transform=ax0.transAxes, fontsize=15)

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=dfs_vlt_dict['MS'][df_agg_key], 
                                  x='endtimeperiod_utc', y='mean_TRS mean_mtrs value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'Monthly Average Voltage (V)', fontsize=25), 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                   seg_line_freq='31D')
ax1.set_ylim([225,250])
vlt_corr = get_correlation_coeff(dfs_vlt_dict['MS'][df_agg_key], 'mean_TRS mean_mtrs value', 
                                 dfs_cir_vlt_dict['MS'][df_key], 'mean_TRS value_mean_cir')
plt.text(1.01, 1.01, f'Corr = {vlt_corr}', transform=ax1.transAxes, fontsize=15)

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=dfs_cir_vlt_dict['MS'][df_key], 
                                  x='date', y='mean_TRS value_mean_cir', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time)

fig_num +=1

In [ ]:
# dfs_cir_kwh_dict['D'][df_key].index==dfs_kwh_dict['D'][df_agg_key].index

## Plots around outage

In [ ]:
expand_time = datetime.timedelta(days=3)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=dfs_kwh_dict['H'][df_agg_key], 
                                  x='endtimeperiod_utc', y='mean_TRS mean_mtrs value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'Hourly Average Consumption (kWh) [expand time = {expand_time}]', fontsize=25), 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                   seg_line_freq='D')

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=dfs_vlt_dict['H'][df_agg_key], 
                                  x='endtimeperiod_utc', y='mean_TRS mean_mtrs value', hue=None, 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'Hourly Average Voltage (V) [expand time = {expand_time}]', fontsize=25), 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                   seg_line_freq='D')
ax1.set_ylim([225,250])
fig_num +=1

In [ ]:
value_col = 'mean_TRS mean_mtrs value'

expand_time = datetime.timedelta(days=3)
fig, ax1 = plt.subplots(1, 1, num=fig_num, figsize=[14, 12])
dfs_kwh_dict['H'][df_agg_key][out_t_beg-expand_time:out_t_end+expand_time].plot(x='endtimeperiod_utc', y=value_col, ax=ax1, color='blue', label='Power', legend=False)
# kwh_mean = dfs_kwh_dict['H'][df_agg_key][out_t_beg-expand_time:out_t_end+expand_time]['value'].mean()
# kwh_std = dfs_kwh_dict['H'][df_agg_key][out_t_beg-expand_time:out_t_end+expand_time]['value'].std()
# ax1.set_ylim([0,kwh_mean+2*kwh_std])

ax2 = ax1.twinx()
dfs_vlt_dict['H'][df_agg_key][out_t_beg-expand_time:out_t_end+expand_time].plot(x='endtimeperiod_utc', y=value_col, ax=ax2, color='red', label='Voltage', legend=False)
vlt_mean = dfs_vlt_dict['H'][df_agg_key][out_t_beg-expand_time:out_t_end+expand_time][value_col].mean()
vlt_std = dfs_vlt_dict['H'][df_agg_key][out_t_beg-expand_time:out_t_end+expand_time][value_col].std()
ax2.set_ylim([vlt_mean-vlt_std,vlt_mean+vlt_std])
ax1.figure.legend(bbox_to_anchor=(1,1), loc='upper left')


In [ ]:
save_figs=False
expand_time = datetime.timedelta(days=3)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
subplots_adjust_args = {'left':0.075, 'right':0.975, 
                        'bottom':0.075, 'top':0.95, 
                        'wspace':0.2, 'hspace':0.35}
plt.subplots_adjust(**subplots_adjust_args)

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=dfs_kwh_dict['H'][df_key], 
                                  x='date', y='mean_TRS value', hue='serialnumber', 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'Hourly Consumption (kWh) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=dfs_kwh_dict['H'][df_agg_key], df_mean_col='mean_TRS mean_mtrs value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True, seg_line_freq='D')

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=dfs_vlt_dict['H'][df_key], 
                                  x='date', y='mean_TRS value', hue='serialnumber', 
                                  out_t_beg=out_t_beg, out_t_end=out_t_end, expand_time=expand_time, 
                                  title_args=dict(label=f'Hourly Voltage (V) [expand time = {expand_time}]', fontsize=25), 
                                  df_mean=dfs_vlt_dict['H'][df_agg_key], df_mean_col='mean_TRS mean_mtrs value', mean_args=None, 
                                  draw_outage_limits=True, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=True, seg_line_freq='D')
ax1.set_ylim([225,250])
#----------------------------------------
if save_figs:
    save_dir = r'C:\Users\s346557\Documents\Presentations\GroupMeetings\20220120\Figures\SampleUsage'
    save_name = 'hourly_consumption_and_voltage.png'
    fig.savefig(os.path.join(save_dir, save_name))
#----------------------------------------    
fig_num +=1

## Plots around no-outage center

In [ ]:
expand_time = datetime.timedelta(days=3)
fig, (ax0,ax1) = plt.subplots(2, 1, num=fig_num, figsize=[14, 12])
plt.subplots_adjust(hspace=0.3)

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=dfs_kwh_dict['H'][df_agg_key], 
                                  x='endtimeperiod_utc', y='mean_TRS mean_mtrs value', hue=None, 
                                  out_t_beg=no_outage_center, out_t_end=no_outage_center, expand_time=expand_time, 
                                  title_args=dict(label=f'Hourly Average Consumption (kWh) [expand time = {expand_time}]', fontsize=25), 
                                  draw_outage_limits=False, 
                                  ylabel_args = dict(ylabel='kWh', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'))

fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=dfs_vlt_dict['H'][df_agg_key], 
                                  x='endtimeperiod_utc', y='mean_TRS mean_mtrs value', hue=None, 
                                  out_t_beg=no_outage_center, out_t_end=no_outage_center, expand_time=expand_time, 
                                  title_args=dict(label=f'Hourly Average Voltage (V) [expand time = {expand_time}]', fontsize=25), 
                                  draw_outage_limits=False, 
                                  ylabel_args = dict(ylabel='Voltage (V)', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'), 
                                  xlabel_args = dict(xlabel='Usage Start Time', fontsize=20, x=0.9, y=0.0, ha='right', va='top'))
fig_num +=1

# -------------------------------------------------------------------
# MODELING AND PREDICTING/FORECASTING
# -------------------------------------------------------------------

In [ ]:
expand_time = datetime.timedelta(days=21)
value_col = 'mean_TRS mean_mtrs value'

In [ ]:
df_kwh_H_mean_cpy = dfs_kwh_dict['H'][df_agg_key].copy()
df_kwh_H_mean_cpy['returns'] = df_kwh_H_mean_cpy[value_col].pct_change(1)*100
df_kwh_H_mean_cpy['sq_returns'] = df_kwh_H_mean_cpy['returns'].mul(df_kwh_H_mean_cpy['returns'])
#-----
df_kwh_no_outage = df_kwh_H_mean_cpy.loc[no_outage_center-expand_time:no_outage_center+expand_time][[value_col, 'returns', 'sq_returns']].copy()
df_kwh_outage = df_kwh_H_mean_cpy.loc[out_t_beg-expand_time:out_t_end+expand_time][[value_col, 'returns', 'sq_returns']].copy()

In [ ]:
df_vlt_H_mean_cpy = dfs_vlt_dict['H'][df_agg_key].copy()
df_vlt_H_mean_cpy['returns'] = df_vlt_H_mean_cpy[value_col].pct_change(1)*100
df_vlt_H_mean_cpy['sq_returns'] = df_vlt_H_mean_cpy['returns'].mul(df_vlt_H_mean_cpy['returns'])
#-----
df_vlt_no_outage = df_vlt_H_mean_cpy.loc[no_outage_center-expand_time:no_outage_center+expand_time][[value_col, 'returns', 'sq_returns']].copy()
df_vlt_outage = df_vlt_H_mean_cpy.loc[out_t_beg-expand_time:out_t_end+expand_time][[value_col, 'returns', 'sq_returns']].copy()

In [ ]:
df_kwh_no_outage[value_col].plot(figsize=(12,5))

In [ ]:
df_vlt_no_outage[value_col].plot(figsize=(12,5))

In [ ]:
df_kwh_no_outage['sq_returns'].plot(figsize=(12,5))

In [ ]:
df_vlt_no_outage['sq_returns'].plot(figsize=(12,5))

In [ ]:
dft.adf_test(df_kwh_no_outage[value_col])

In [ ]:
dft.adf_test(df_vlt_no_outage[value_col])

In [ ]:
run_auto_arima = False
if run_auto_arima:
    model_kwh_auto = auto_arima(df_kwh_no_outage[value_col], seasonal=True, m=24)
    print(model_kwh_auto.summary())
    pred_kwh = model_kwh_auto.predict(n_periods=48)
    dft.adf_test(pd.DataFrame(model_kwh_auto.resid()))
    plot_acf(pd.DataFrame(model_kwh_auto.resid()));
    plot_pacf(pd.DataFrame(model_kwh_auto.resid()));
    
    model_vlt_auto = auto_arima(df_vlt_no_outage[value_col], seasonal=True, m=24)
    print(model_vlt_auto.summary())
    pred_vlt = model_vlt_auto.predict(n_periods=48)

In [ ]:
df_kwh_no_outage['returns'].plot(figsize=(12,6))

In [ ]:
df_vlt_no_outage['returns'].plot(figsize=(12,6))

In [ ]:
plot_pacf(df_kwh_no_outage['returns']);

In [ ]:
plot_pacf(df_kwh_no_outage['sq_returns']);

In [ ]:
plot_pacf(df_vlt_no_outage['returns']);

In [ ]:
plot_pacf(df_vlt_no_outage['sq_returns']);

In [ ]:
split_pct = 0.80
split_date = df_vlt_no_outage.index[round(split_pct*len(df_vlt_no_outage))]

In [ ]:
model_arch_1 = arch_model(df_vlt_no_outage['returns'], mean='Constant', vol='ARCH', p=1)
results_arch_1 = model_arch_1.fit(last_obs = split_date)
results_arch_1.summary()

In [ ]:
pred_arch_1 = results_arch_1.forecast(horizon=1, align='target')

In [ ]:
pred_arch_1.residual_variance[split_date:].plot(figsize=(12,5), color='red', zorder=2)
df_vlt_no_outage[split_date:]['returns'].abs().plot(color='blue', zorder=1)

In [ ]:
model_garch_1_1 = arch_model(df_vlt_no_outage['returns'], mean='Constant', vol='GARCH', p=1, q=1)
results_garch_1_1 = model_garch_1_1.fit(last_obs = split_date)
results_garch_1_1.summary()

In [ ]:
dfs_kwh_dict['15T'][df_agg_key]

In [ ]:
value_col = 'mean_mtrs value'
df_kwh_15T_mean_cpy = dfs_kwh_dict['15T'][df_agg_key].copy()
df_kwh_15T_mean_cpy['returns'] = df_kwh_15T_mean_cpy[value_col].pct_change(1)*100
df_kwh_15T_mean_cpy['sq_returns'] = df_kwh_15T_mean_cpy['returns'].mul(df_kwh_15T_mean_cpy['returns'])
#-----
df_kwh_15T_mean_no_outage = df_kwh_15T_mean_cpy.loc[no_outage_center-expand_time:no_outage_center+expand_time][[value_col, 'returns', 'sq_returns']].copy()
df_kwh_15T_mean_outage = df_kwh_15T_mean_cpy.loc[out_t_beg-expand_time:out_t_end+expand_time][[value_col, 'returns', 'sq_returns']].copy()

In [ ]:
df_kwh_15T_cpy = df_kwh_15T.copy()
df_kwh_15T_cpy['returns'] = None
df_kwh_15T_cpy['returns'] = df_kwh_15T_cpy.groupby('serialnumber')['value'].pct_change(1)*100
df_kwh_15T_cpy['sq_returns'] = df_kwh_15T_cpy['returns'].mul(df_kwh_15T_cpy['returns'])
#-----
df_kwh_15T_no_outage = df_kwh_15T_cpy.loc[no_outage_center-expand_time:no_outage_center+expand_time].copy()
df_kwh_15T_outage = df_kwh_15T_cpy.loc[out_t_beg-expand_time:out_t_end+expand_time].copy()

In [ ]:
df_kwh_15T_mean_no_outage['sq_returns'].plot(figsize=(20,5))
plt.title("15T Mean Volatility - No Outage", size = 24)

In [ ]:
out_t_beg

In [ ]:
df_kwh_15T_mean_outage['2020-06-15 00:00:00':'2020-06-21 14:52:00']['sq_returns'].plot(figsize=(20,5))
plt.title("15T Mean Volatility - Outage (BEFORE)", size = 24)

In [ ]:
df_kwh_15T_mean_outage[:out_t_beg]['sq_returns'].plot(figsize=(20,5))
plt.title("15T Mean Volatility - Outage (BEFORE)", size = 24)

In [ ]:
df_kwh_15T_mean_outage['2020-06-21 16:52:00':'2020-06-28 00:00:00']['sq_returns'].plot(figsize=(20,5))
plt.title("15T Mean Volatility - Outage (AFTER)", size = 24)

In [ ]:
out_t_end

In [ ]:
ax=df_kwh_15T_mean_outage[out_t_end:]['sq_returns'].plot(figsize=(20,5))
plt.title("15T Mean Volatility - Outage (AFTER)", size = 24)
#ax.set_ylim([0, 70000])
ax.set_ylim([0, 7500])

In [ ]:
df_kwh_15T_mean_outage['sq_returns'].mean()

In [ ]:
ax=df_kwh_15T_mean_outage['2020-06-15 00:00:00':'2020-06-28 00:00:00']['sq_returns'].plot(figsize=(20,5))
plt.title("15T Mean Volatility - Outage", size = 24)
#ax.set_ylim([0, 70000])
ax.set_ylim([0, 7500])
ax.axvline(out_t_beg, color='red')
ax.axvline(out_t_end, color='lawngreen')

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[20, 5])
plt.subplots_adjust(hspace=0.3)
fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T_no_outage, 
                                  x='endtimeperiod_utc', y='sq_returns', hue='serialnumber', 
                                  out_t_beg=no_outage_center, out_t_end=no_outage_center, expand_time=expand_time, 
                                  title_args=dict(label=f'15T Volatility - No Outage [expand time = {expand_time}]', fontsize=25), 
                                  draw_outage_limits=False, 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=False)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[20, 5])
plt.subplots_adjust(hspace=0.3)
fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T_outage, 
                                  x='endtimeperiod_utc', y='sq_returns', hue='serialnumber', 
                                  out_t_beg=pd.to_datetime('2020-06-15 00:00:00'), out_t_end=pd.to_datetime('2020-06-21 14:52:00'), expand_time=datetime.timedelta(days=0), 
                                  title_args=dict(label=f'15T Volatility - Outage (BEFORE)', fontsize=25), 
                                  draw_outage_limits=False, 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=False)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[20, 5])
plt.subplots_adjust(hspace=0.3)
fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T_outage, 
                                  x='endtimeperiod_utc', y='sq_returns', hue='serialnumber', 
                                  out_t_beg=pd.to_datetime('2020-06-21 16:52:00'), out_t_end=pd.to_datetime('2020-06-25 00:00:00'), expand_time=datetime.timedelta(days=0), 
                                  title_args=dict(label=f'15T Volatility - Outage (AFTER)', fontsize=25), 
                                  draw_outage_limits=False, 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=False)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[20, 5])
plt.subplots_adjust(hspace=0.3)
fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T_outage, 
                                  x='endtimeperiod_utc', y='sq_returns', hue='serialnumber', 
                                  out_t_beg=pd.to_datetime('2020-06-15 00:00:00'), out_t_end=pd.to_datetime('2020-06-25 00:00:00'), expand_time=datetime.timedelta(days=0), 
                                  title_args=dict(label=f'15T Volatility - Outage', fontsize=25), 
                                  draw_outage_limits=False, 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=False)
ax0.set_ylim([0, 1.2e7])
ax0.axvline(out_t_beg, color='red')
ax0.axvline(out_t_end, color='lawngreen')

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[20, 5])
plt.subplots_adjust(hspace=0.3)
fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T_outage[df_kwh_15T_outage['sq_returns']>0], 
                                  x='endtimeperiod_utc', y='sq_returns', hue='serialnumber', 
                                  out_t_beg=pd.to_datetime('2020-06-15 00:00:00'), out_t_end=pd.to_datetime('2020-06-25 00:00:00'), expand_time=datetime.timedelta(days=0), 
                                  title_args=dict(label=f'15T Volatility - Outage', fontsize=25), 
                                  draw_outage_limits=False, 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=False)
ax0.set_ylim([0, 1.2e7])
ax0.axvline(out_t_beg, color='red')
ax0.axvline(out_t_end, color='lawngreen')
ax0.set_ylim(0.1, 1.2e7)
ax0.set_yscale('log')

In [ ]:
#tmp_df_kwh_H_outage = df_kwh_15T_outage.resample('H').agg({'sq_returns':'mean'})
tmp_df_kwh_H_outage = df_kwh_15T_outage.groupby([pd.Grouper(freq='H')] + ['serialnumber']).agg({'sq_returns':'mean'})

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[20, 5])
plt.subplots_adjust(hspace=0.3)
fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=tmp_df_kwh_H_outage[tmp_df_kwh_H_outage['sq_returns']>0], 
                                  x='time_idx', y='sq_returns', hue='serialnumber', 
                                  out_t_beg=pd.to_datetime('2020-06-15 00:00:00'), out_t_end=pd.to_datetime('2020-06-25 00:00:00'), expand_time=datetime.timedelta(days=0), 
                                  title_args=dict(label=f'Hourly Volatility - Outage', fontsize=25), 
                                  draw_outage_limits=False, 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=False)
ax0.set_ylim([0, 1.2e7])
ax0.axvline(out_t_beg, color='red')
ax0.axvline(out_t_end, color='lawngreen')
ax0.set_ylim(0.1, 1.2e7)
ax0.set_yscale('log')

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[20, 5])
plt.subplots_adjust(hspace=0.3)
fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=tmp_df_kwh_H_outage, 
                                  x='time_idx', y='sq_returns', hue='serialnumber', 
                                  out_t_beg=pd.to_datetime('2020-06-15 00:00:00'), out_t_end=pd.to_datetime('2020-06-25 00:00:00'), expand_time=datetime.timedelta(days=0), 
                                  title_args=dict(label=f'Hourly Volatility - Outage', fontsize=25), 
                                  draw_outage_limits=False, 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=False)
ax0.set_ylim([0, 1.2e7])
ax0.axvline(out_t_beg, color='red')
ax0.axvline(out_t_end, color='lawngreen')

In [ ]:
save_figs=False
tmp_df_kwh_H_outage = df_kwh_15T_outage.groupby([pd.Grouper(freq='H')] + ['serialnumber']).agg({'sq_returns':'mean'})

fig, (ax0, ax1, ax2) = plt.subplots(3, 1, num=fig_num, figsize=[14, 18])
subplots_adjust_args = {'left':0.075, 'right':0.975, 
                        'bottom':0.075, 'top':0.95, 
                        'wspace':0.2, 'hspace':0.35}
plt.subplots_adjust(**subplots_adjust_args)

fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T_outage, 
                                  x='endtimeperiod_utc', y='sq_returns', hue='serialnumber', 
                                  out_t_beg=pd.to_datetime('2020-06-15 00:00:00'), out_t_end=pd.to_datetime('2020-06-25 00:00:00'), expand_time=datetime.timedelta(days=0), 
                                  title_args=dict(label=f'15T Volatility - Outage', fontsize=25), 
                                  draw_outage_limits=False, 
                                  xlabel_args = dict(xlabel='Datetime', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                   ylabel_args = dict(ylabel=f'$\Delta^{2}$', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'),
                                  draw_without_hue_also=False)
ax0.set_ylim([0, 1.2e7])
ax0.axvline(out_t_beg, color='red')
ax0.axvline(out_t_end, color='lawngreen')
#------------------------------------------
fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=tmp_df_kwh_H_outage, 
                                  x='time_idx', y='sq_returns', hue='serialnumber', 
                                  out_t_beg=pd.to_datetime('2020-06-15 00:00:00'), out_t_end=pd.to_datetime('2020-06-25 00:00:00'), expand_time=datetime.timedelta(days=0), 
                                  title_args=dict(label=f'Hourly Volatility - Outage', fontsize=25), 
                                  draw_outage_limits=False, 
                                  xlabel_args = dict(xlabel='Datetime', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                   ylabel_args = dict(ylabel=f'$\Delta^{2}$', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'),
                                  draw_without_hue_also=False)
ax1.set_ylim([0, 1.2e7])
ax1.axvline(out_t_beg, color='red')
ax1.axvline(out_t_end, color='lawngreen')
#------------------------------------------
fig,ax2 = AMINonVee.plot_usage_around_outage(fig, ax2, data=tmp_df_kwh_H_outage[tmp_df_kwh_H_outage['sq_returns']>0], 
                                  x='time_idx', y='sq_returns', hue='serialnumber', 
                                  out_t_beg=pd.to_datetime('2020-06-15 00:00:00'), out_t_end=pd.to_datetime('2020-06-25 00:00:00'), expand_time=datetime.timedelta(days=0), 
                                  title_args=dict(label=f'Hourly Volatility - Outage', fontsize=25), 
                                  draw_outage_limits=False, 
                                  xlabel_args = dict(xlabel='Datetime', fontsize=20, x=0.9, y=0.0, ha='right', va='top'), 
                                   ylabel_args = dict(ylabel=f'$\Delta^{2}$', fontsize=20, x=0.0, y=0.8, ha='left', va='bottom'),
                                  draw_without_hue_also=False)
ax2.set_ylim([0, 1.2e7])
ax2.axvline(out_t_beg, color='red')
ax2.axvline(out_t_end, color='lawngreen')
ax2.set_ylim(0.1, 1.2e7)
ax2.set_yscale('log')
#----------------------------------------
if save_figs:
    save_dir = r'C:\Users\s346557\Documents\Presentations\GroupMeetings\20220120\Figures\SampleUsage'
    save_name = 'volatility.png'
    fig.savefig(os.path.join(save_dir, save_name))
#----------------------------------------    

In [ ]:
df_kwh_15T_mean_outage['endtimeperiod_utc'] = df_kwh_15T_mean_outage.index
fig, (ax0, ax1) = plt.subplots(2, 1, num=fig_num, figsize=[20, 10], sharex=True)
plt.subplots_adjust(hspace=0.3)
# df_kwh_15T_mean_outage['2020-06-15 00:00:00':'2020-06-28 00:00:00']['sq_returns'].plot(figsize=(20,5), ax=ax0)
sns.lineplot(ax=ax0, data=df_kwh_15T_mean_outage['2020-06-23 16:00:00':'2020-06-23 20:00:00'], 
             x='endtimeperiod_utc', y='sq_returns', 
             palette='colorblind')
ax0.set_title("15T Mean Volatility - Outage", size = 24)
#ax0.set_ylim([0, 70000])
# ax0.axvline(out_t_beg, color='red')
# ax0.axvline(out_t_end, color='lawngreen')

sns.lineplot(ax=ax1, data=df_kwh_15T_outage['2020-06-23 16:00:00':'2020-06-23 20:00:00'], 
             x='endtimeperiod_utc', y='sq_returns', hue='serialnumber', 
             palette='colorblind')

# fig,ax1 = AMINonVee.plot_usage_around_outage(fig, ax1, data=df_kwh_15T_outage, 
#                                   x='endtimeperiod_utc', y='sq_returns', hue='serialnumber', 
#                                   out_t_beg=pd.to_datetime('2020-06-15 00:00:00'), out_t_end=pd.to_datetime('2020-06-28 00:00:00'), expand_time=datetime.timedelta(days=0), 
#                                   title_args=dict(label=f'15T Volatility - Outage', fontsize=25), 
#                                   draw_outage_limits=False, 
#                                   xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
#                                   draw_without_hue_also=False)
ax1.set_ylim([0, 1.2e7])
# ax1.axvline(out_t_beg, color='red')
# ax1.axvline(out_t_end, color='lawngreen')

In [ ]:
# df_kwh_15T_outage.loc['2020-06-23 18:30:00']['sq_returns'].mean()

In [ ]:
# df_kwh_15T_mean_outage.loc['2020-06-23 18:30:00']

In [ ]:
df_kwh_15T_mean_no_outage['returns'].plot(figsize=(20,5))
plt.title("15T Mean Deltas - No Outage", size = 24)

In [ ]:
df_kwh_15T_mean_outage['2020-06-15 00:00:00':'2020-06-21 14:52:00']['returns'].plot(figsize=(20,5))
plt.title("15T Mean Deltas - Outage (BEFORE)", size = 24)

In [ ]:
df_kwh_15T_mean_outage['2020-06-21 16:52:00':'2020-06-28 00:00:00']['returns'].plot(figsize=(20,5))
plt.title("15T Mean Deltas - Outage (AFTER)", size = 24)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[20, 5])
plt.subplots_adjust(hspace=0.3)
fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T_no_outage, 
                                  x='endtimeperiod_utc', y='returns', hue='serialnumber', 
                                  out_t_beg=no_outage_center, out_t_end=no_outage_center, expand_time=expand_time, 
                                  title_args=dict(label=f'15T Deltas - No Outage [expand time = {expand_time}]', fontsize=25), 
                                  draw_outage_limits=False, 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=False)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[20, 5])
plt.subplots_adjust(hspace=0.3)
fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T_outage, 
                                  x='endtimeperiod_utc', y='returns', hue='serialnumber', 
                                  out_t_beg=pd.to_datetime('2020-06-15 00:00:00'), out_t_end=pd.to_datetime('2020-06-21 14:52:00'), expand_time=datetime.timedelta(days=0), 
                                  title_args=dict(label=f'15T Deltas - Outage (BEFORE)', fontsize=25), 
                                  draw_outage_limits=False, 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=False)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[20, 5])
plt.subplots_adjust(hspace=0.3)
fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T_outage, 
                                  x='endtimeperiod_utc', y='returns', hue='serialnumber', 
                                  out_t_beg=pd.to_datetime('2020-06-21 16:52:00'), out_t_end=pd.to_datetime('2020-06-25 00:00:00'), expand_time=datetime.timedelta(days=0), 
                                  title_args=dict(label=f'15T Deltas - Outage (AFTER)', fontsize=25), 
                                  draw_outage_limits=False, 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=False)

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[20, 5])
plt.subplots_adjust(hspace=0.3)
fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T_outage, 
                                  x='endtimeperiod_utc', y='returns', hue='serialnumber', 
                                  out_t_beg=pd.to_datetime('2020-06-15 00:00:00'), out_t_end=pd.to_datetime('2020-06-28 00:00:00'), expand_time=datetime.timedelta(days=0), 
                                  title_args=dict(label=f'15T Deltas - Outage', fontsize=25), 
                                  draw_outage_limits=False, 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=False)
ax0.set_ylim([-200, 7000])
ax0.axvline(out_t_beg, color='red')
ax0.axvline(out_t_end, color='lawngreen')

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[20, 5])
plt.subplots_adjust(hspace=0.3)
fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T_outage, 
                                  x='endtimeperiod_utc', y='value', hue='serialnumber', 
                                  out_t_beg=pd.to_datetime('2020-06-15 00:00:00'), out_t_end=pd.to_datetime('2020-06-28 00:00:00'), expand_time=datetime.timedelta(days=0), 
                                  title_args=dict(label=f'15T Usage - Outage', fontsize=25), 
                                  draw_outage_limits=False, 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=False, seg_line_freq='D')
#ax0.set_ylim([-200, 7000])
ax0.axvline(out_t_beg, color='red')
ax0.axvline(out_t_end, color='lawngreen')

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[20, 5])
plt.subplots_adjust(hspace=0.3)
fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T_mean_outage, 
                                  x='endtimeperiod_utc', y='mean_mtrs value', hue=None, 
                                  out_t_beg=pd.to_datetime('2020-06-15 00:00:00'), out_t_end=pd.to_datetime('2020-06-28 00:00:00'), expand_time=datetime.timedelta(days=0), 
                                  title_args=dict(label=f'15T Usage - Outage', fontsize=25), 
                                  draw_outage_limits=False, 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=False, seg_line_freq='D')
#ax0.set_ylim([-200, 7000])
ax0.axvline(out_t_beg, color='red')
ax0.axvline(out_t_end, color='lawngreen')

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[20, 5])
plt.subplots_adjust(hspace=0.3)
fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T_mean_outage, 
                                  x='endtimeperiod_utc', y='mean_mtrs value', hue=None, 
                                  out_t_beg=pd.to_datetime('2020-06-19 00:00:00'), out_t_end=pd.to_datetime('2020-06-24 00:00:00'), expand_time=datetime.timedelta(days=0), 
                                  title_args=dict(label=f'15T Usage - Outage', fontsize=25), 
                                  draw_outage_limits=False, 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=False, seg_line_freq='D')
#ax0.set_ylim([-200, 7000])
ax0.axvline(out_t_beg, color='red')
ax0.axvline(out_t_end, color='lawngreen')

In [ ]:
fig, ax0 = plt.subplots(1, 1, num=fig_num, figsize=[20, 5])
plt.subplots_adjust(hspace=0.3)
fig,ax0 = AMINonVee.plot_usage_around_outage(fig, ax0, data=df_kwh_15T_outage, 
                                  x='endtimeperiod_utc', y='value', hue='serialnumber', 
                                  out_t_beg=pd.to_datetime('2020-06-19 00:00:00'), out_t_end=pd.to_datetime('2020-06-24 00:00:00'), expand_time=datetime.timedelta(days=0), 
                                  title_args=dict(label=f'15T Usage - Outage', fontsize=25), 
                                  draw_outage_limits=False, 
                                  xlabel_args = dict(xlabel='', fontsize=0, x=0.9, y=0.0, ha='right', va='top'), 
                                  draw_without_hue_also=False, seg_line_freq='D')
#ax0.set_ylim([-200, 7000])
ax0.axvline(out_t_beg, color='red')
ax0.axvline(out_t_end, color='lawngreen')

In [ ]:
value_col = 'mean_TRS mean_mtrs value'
df_kwh_H_mean_cpy = dfs_kwh_dict['H'][df_agg_key].copy()
df_kwh_H_mean_cpy['returns'] = df_kwh_H_mean_cpy[value_col].pct_change(1)*100
df_kwh_H_mean_cpy['sq_returns'] = df_kwh_H_mean_cpy['returns'].mul(df_kwh_H_mean_cpy['returns'])
#-----
df_kwh_H_mean_no_outage = df_kwh_H_mean_cpy.loc[no_outage_center-expand_time:no_outage_center+expand_time][[value_col, 'returns', 'sq_returns']].copy()
df_kwh_H_mean_outage = df_kwh_H_mean_cpy.loc[out_t_beg-expand_time:out_t_end+expand_time][[value_col, 'returns', 'sq_returns']].copy()

In [ ]:
df_kwh_H_mean_outage[value_col].plot(figsize=(20,10))
df_kwh_H_mean_outage[value_col].rolling(24, center=True).mean().plot()
df_kwh_H_mean_outage[value_col].rolling(24*7, center=True).mean().plot()

In [ ]:
df_kwh_H_mean_no_outage[value_col].plot(figsize=(20,10))
df_kwh_H_mean_no_outage[value_col].rolling(24, center=True).mean().plot()
df_kwh_H_mean_no_outage[value_col].rolling(24*7, center=True).mean().plot()

In [ ]:
train = df_kwh_H_mean_outage[:'2020-06-21 00:00:00'][[value_col]]
test = df_kwh_H_mean_outage['2020-06-21 00:00:00':][[value_col]]

# train = df_kwh_H_mean_outage[:'2020-07-05 00:00:00'][['value']]
# test = df_kwh_H_mean_outage['2020-07-05 00:00:00':][['value']]

# model_auto = auto_arima(train['value'], seasonal=True, m=24)
# print(model_auto.summary())

model = SARIMAX(train[value_col], order=(2, 0, 0), seasonal_order=(1, 0, [1], 24))
results = model.fit()
print(results.summary())

predictions = results.predict(len(train), len(df_kwh_H_mean_outage), typ='levels')
all_pred = results.get_prediction(df_kwh_H_mean_outage.index[0], df_kwh_H_mean_outage.index[-1])

seg_min = df_kwh_H_mean_outage.index[0].round('D')
seg_max = pd.to_datetime(df_kwh_H_mean_outage.index[-1]).round('D')
all_segs_D = pd.date_range(seg_min, seg_max, freq='D')
all_segs_W = pd.date_range(seg_min, seg_max, freq='W')

fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(20,30))
#--------------------
df_kwh_H_mean_outage[value_col].plot(figsize=(20,10), ax=ax1)
predictions.plot(ax=ax1)
ax1.axvline(df_kwh_H_mean_outage.index[len(train)], color='purple')
ax1.axvline(out_t_beg, color='red')
ax1.axvline(out_t_end, color='lawngreen')
for seg in all_segs_D:
    ax1.axvline(seg, color='black', linestyle='--', linewidth=0.5);
for seg in all_segs_W:
    ax1.axvline(seg, color='black', linestyle='--', linewidth=2.0);
#--------------------
df_kwh_H_mean_outage[value_col].plot(figsize=(20,10), ax=ax2)
all_pred.predicted_mean.plot(ax=ax2)
ax2.axvline(out_t_beg, color='red')
ax2.axvline(out_t_end, color='lawngreen')
for seg in all_segs_D:
    ax2.axvline(seg, color='black', linestyle='--', linewidth=0.5);
for seg in all_segs_W:
    ax2.axvline(seg, color='black', linestyle='--', linewidth=2.0);
#--------------------
df_kwh_H_mean_outage[value_col].plot(figsize=(20,10), ax=ax3)
results.fittedvalues.plot(ax=ax3)
ax3.axvline(out_t_beg, color='red')
ax3.axvline(out_t_end, color='lawngreen')
for seg in all_segs_D:
    ax3.axvline(seg, color='black', linestyle='--', linewidth=0.5);
for seg in all_segs_W:
    ax3.axvline(seg, color='black', linestyle='--', linewidth=2.0);

In [ ]:
train = df_kwh_H_mean_no_outage[:'2020-08-23 04:00:00'][[value_col]]
test = df_kwh_H_mean_no_outage['2020-08-23 04:00:00':][[value_col]]

# model_auto = auto_arima(train[value_col], seasonal=True, m=24)
# print(model_auto.summary())

model = SARIMAX(train[value_col], order=(2, 0, 0), seasonal_order=(1, 0, [1], 24))
results = model.fit()
print(results.summary())

predictions = results.predict(len(train), len(df_kwh_H_mean_no_outage), typ='levels')
all_pred = results.get_prediction(df_kwh_H_mean_no_outage.index[0], df_kwh_H_mean_no_outage.index[-1])

seg_min = df_kwh_H_mean_no_outage.index[0].round('D')
seg_max = pd.to_datetime(df_kwh_H_mean_no_outage.index[-1]).round('D')
all_segs_D = pd.date_range(seg_min, seg_max, freq='D')
all_segs_W = pd.date_range(seg_min, seg_max, freq='W')

fig, (ax1, ax2, ax3) = plt.subplots(3,1, figsize=(20,30))
#--------------------
df_kwh_H_mean_no_outage[value_col].plot(figsize=(20,10), ax=ax1)
predictions.plot(ax=ax1)
ax1.axvline(df_kwh_H_mean_no_outage.index[len(train)], color='purple')
for seg in all_segs_D:
    ax1.axvline(seg, color='black', linestyle='--', linewidth=0.5);
for seg in all_segs_W:
    ax1.axvline(seg, color='black', linestyle='--', linewidth=2.0);
#--------------------
df_kwh_H_mean_no_outage[value_col].plot(figsize=(20,10), ax=ax2)
all_pred.predicted_mean.plot(ax=ax2)
for seg in all_segs_D:
    ax2.axvline(seg, color='black', linestyle='--', linewidth=0.5);
for seg in all_segs_W:
    ax2.axvline(seg, color='black', linestyle='--', linewidth=2.0);
#--------------------
df_kwh_H_mean_no_outage[value_col].plot(figsize=(20,10), ax=ax3)
results.fittedvalues.plot(ax=ax3)
for seg in all_segs_D:
    ax3.axvline(seg, color='black', linestyle='--', linewidth=0.5);
for seg in all_segs_W:
    ax3.axvline(seg, color='black', linestyle='--', linewidth=2.0);

In [ ]:
save_figs=False
train = df_kwh_H_mean_outage[:'2020-06-21 00:00:00'][[value_col]]
test = df_kwh_H_mean_outage['2020-06-21 00:00:00':][[value_col]]

# train = df_kwh_H_mean_outage[:'2020-07-05 00:00:00'][['value']]
# test = df_kwh_H_mean_outage['2020-07-05 00:00:00':][['value']]

# model_auto = auto_arima(train['value'], seasonal=True, m=24)
# print(model_auto.summary())

model = SARIMAX(train[value_col], order=(2, 0, 0), seasonal_order=(1, 0, [1], 24))
results = model.fit()
print(results.summary())

predictions = results.predict(len(train), len(df_kwh_H_mean_outage), typ='levels')
all_pred = results.get_prediction(df_kwh_H_mean_outage.index[0], df_kwh_H_mean_outage.index[-1])

seg_min = df_kwh_H_mean_outage.index[0].round('D')
seg_max = pd.to_datetime(df_kwh_H_mean_outage.index[-1]).round('D')
all_segs_D = pd.date_range(seg_min, seg_max, freq='D')
all_segs_W = pd.date_range(seg_min, seg_max, freq='W')

fig, ax1 = plt.subplots(1,1, figsize=(14,6))
subplots_adjust_args = {'left':0.075, 'right':0.975, 
                        'bottom':0.25, 'top':0.925, 
                        'wspace':0.2, 'hspace':0.35}
plt.subplots_adjust(**subplots_adjust_args)
#--------------------
df_kwh_H_mean_outage[value_col].plot(ax=ax1, label='Data', color='goldenrod')
predictions.plot(ax=ax1, label='Predictions (SARIMAX)', color='C0')
ax1.axvline(df_kwh_H_mean_outage.index[len(train)], color='purple')
# ax1.axvline(out_t_beg, color='red')
# ax1.axvline(out_t_end, color='lawngreen')
for seg in all_segs_D:
    ax1.axvline(seg, color='black', linestyle='--', linewidth=0.5);
for seg in all_segs_W:
    ax1.axvline(seg, color='black', linestyle='--', linewidth=2.0);
    
ax1.legend(fontsize=20)
ax1.set_title(label=f'Hourly Average Consumption (kWh)', fontsize=25)
ax1.set_xlabel(xlabel='Datetime', fontsize=30, x=0.9, y=0.0, ha='right', va='top')
ax1.set_ylabel(ylabel='kWh', fontsize=30, x=0.0, y=0.8, ha='left', va='bottom')
ax1.tick_params(axis='x', labelrotation=45, labelsize=20)
ax1.tick_params(axis='x', which='minor', labelsize=20)
ax1.tick_params(axis='y', labelsize=20)

#----------------------------------------
if save_figs:
    save_dir = r'C:\Users\s346557\Documents\Presentations\GroupMeetings\20220120\Figures\SampleUsage'
    save_name = 'SARIMAX_bad.png'
    fig.savefig(os.path.join(save_dir, save_name))
#---------------------------------------- 

In [ ]:
save_figs=False
train = df_kwh_H_mean_no_outage[:'2020-08-23 04:00:00'][[value_col]]
test = df_kwh_H_mean_no_outage['2020-08-23 04:00:00':][[value_col]]

# model_auto = auto_arima(train[value_col], seasonal=True, m=24)
# print(model_auto.summary())

model = SARIMAX(train[value_col], order=(2, 0, 0), seasonal_order=(1, 0, [1], 24))
results = model.fit()
print(results.summary())

predictions = results.predict(len(train), len(df_kwh_H_mean_no_outage), typ='levels')
all_pred = results.get_prediction(df_kwh_H_mean_no_outage.index[0], df_kwh_H_mean_no_outage.index[-1])

seg_min = df_kwh_H_mean_no_outage.index[0].round('D')
seg_max = pd.to_datetime(df_kwh_H_mean_no_outage.index[-1]).round('D')
all_segs_D = pd.date_range(seg_min, seg_max, freq='D')
all_segs_W = pd.date_range(seg_min, seg_max, freq='W')

fig, ax1 = plt.subplots(1,1, figsize=(14,6))
subplots_adjust_args = {'left':0.075, 'right':0.975, 
                        'bottom':0.25, 'top':0.925, 
                        'wspace':0.2, 'hspace':0.35}
plt.subplots_adjust(**subplots_adjust_args)
#--------------------
df_kwh_H_mean_no_outage[value_col].plot(ax=ax1, label='Data', color='goldenrod')
predictions.plot(ax=ax1, label='Predictions (SARIMAX)', color='C0')
ax1.axvline(df_kwh_H_mean_no_outage.index[len(train)], color='purple')

for seg in all_segs_D:
    ax1.axvline(seg, color='black', linestyle='--', linewidth=0.5);
for seg in all_segs_W:
    ax1.axvline(seg, color='black', linestyle='--', linewidth=2.0);

ax1.legend(fontsize=20)
ax1.set_title(label=f'Hourly Average Consumption (kWh)', fontsize=25)
ax1.set_xlabel(xlabel='Datetime', fontsize=30, x=0.9, y=0.0, ha='right', va='top')
ax1.set_ylabel(ylabel='kWh', fontsize=30, x=0.0, y=0.8, ha='left', va='bottom')
ax1.tick_params(axis='x', labelrotation=45, labelsize=20)
ax1.tick_params(axis='x', which='minor', labelsize=20)
ax1.tick_params(axis='y', labelsize=20)
#----------------------------------------
if save_figs:
    save_dir = r'C:\Users\s346557\Documents\Presentations\GroupMeetings\20220120\Figures\SampleUsage'
    save_name = 'SARIMAX_good.png'
    fig.savefig(os.path.join(save_dir, save_name))
#---------------------------------------- 

In [ ]:
from alibi_detect.od import SpectralResidual
from alibi_detect.datasets import fetch_kdd

In [ ]:
intrusions = fetch_kdd()

In [ ]:
pd.Series(intrusions['data'][:, 0]).plot()

In [ ]:
od = SpectralResidual(
    threshold=1.,
    window_amp=20,
    window_local=20,
    n_est_points=10,
    n_grad_points=5
)

In [ ]:
scores = od.score(intrusions['data'][:, 0])

In [ ]:
scores

In [ ]:
# ax = df_kwh_H_mean_outage['value'].plot(figsize=(20,10))
# ax2 = ax.twinx()
# ax = pd.Series(scores, name='scores').plot(ax=ax2, color='red')

ax = pd.Series(intrusions['data'][:, 0], name='data').plot(legend=False, figsize=(20,10))
ax2 = ax.twinx()
ax = pd.Series(scores, name='scores').plot(ax=ax2, color='red', legend=False)
ax.figure.legend(bbox_to_anchor=(1,1), loc='upper left')

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import tensorflow as tf

from alibi_detect.od import OutlierProphet
from alibi_detect.utils.fetching import fetch_detector
from alibi_detect.utils.saving import save_detector, load_detector

In [ ]:
n_prophet = 910
# d = {'ds': df['Date Time'][:n_prophet], 'y': df['T (degC)'][:n_prophet]}

In [ ]:
df_kwh_H_mean_outage.iloc[:n_prophet][value_col]

In [ ]:
df_T = df_kwh_H_mean_outage.iloc[:n_prophet][value_col].copy()
df_T = df_T.reset_index()
df_T = df_T.rename(columns={'time_idx':'ds', value_col:'y'})
df_T.plot(x='ds', y='y', figsize=(20,10), color='goldenrod')

od = OutlierProphet(threshold=0.99)
od.fit(df_T)

In [ ]:
df_T_test = df_kwh_H_mean_outage.iloc[n_prophet:][value_col].copy()
df_T_test = df_T_test.reset_index()
df_T_test = df_T_test.rename(columns={'time_idx':'ds', value_col:'y'})

od_preds = od.predict(
    df_T_test,
    return_instance_score=True,
    return_forecast=True
)
print(od_preds['data'].keys())
od_preds['data']['forecast'].plot(x='ds', y='yhat', figsize=(20,5));

In [ ]:
n_to_pred = 100
future = od.model.make_future_dataframe(periods=n_to_pred, freq='H', include_history=True)
forecast = od.model.predict(future)
fig = od.model.plot(forecast)
# fig = od.model.plot_components(forecast)

In [ ]:
save_figs = False
fig = od.model.plot_components(forecast)
#----------------------------------------
if save_figs:
    save_dir = r'C:\Users\s346557\Documents\Presentations\GroupMeetings\20220120\Figures\SampleUsage'
    save_name = 'fb_prophet_2.png'
    fig.savefig(os.path.join(save_dir, save_name))
#---------------------------------------- 

In [ ]:
save_figs = False

fig, ax = plt.subplots(1, 1, num=fig_num, figsize=[14, 6])
subplots_adjust_args = {'left':0.075, 'right':0.975, 
                        'bottom':0.15, 'top':0.95, 
                        'wspace':0.2, 'hspace':0.35}
plt.subplots_adjust(**subplots_adjust_args)

forecast_to_plot = forecast.iloc[:len(df_T)]
forecast_to_plot['y'] = df_T['y']
forecast_to_plot.plot(ax=ax, x='ds', y=['y', 'yhat'], figsize=(20,10), color=['goldenrod', 'C0'], label=['Data', 'Fit (OutlierProphet)'])
ax.fill_between(forecast_to_plot['ds'].values, 
                forecast_to_plot['yhat_lower'], 
                forecast_to_plot['yhat_upper'], alpha=0.2)
ax.legend(fontsize=20)
ax.set_title(label=f'Hourly Average Consumption (kWh)', fontsize=25)
ax.set_xlabel(xlabel='Datetime', fontsize=30, x=0.9, y=0.0, ha='right', va='top')
ax.set_ylabel(ylabel='kWh', fontsize=30, x=0.0, y=0.8, ha='left', va='bottom')
ax.tick_params(axis='x', labelrotation=45, labelsize=20)
ax.tick_params(axis='x', which='minor', labelsize=20)
ax.tick_params(axis='y', labelsize=20)
#----------------------------------------
if save_figs:
    save_dir = r'C:\Users\s346557\Documents\Presentations\GroupMeetings\20220120\Figures\SampleUsage'
    save_name = 'fb_prophet_1.png'
    fig.savefig(os.path.join(save_dir, save_name))
#---------------------------------------- 

In [ ]:
ax=od_preds['data']['forecast'].plot(x='ds', y=['yhat'], figsize=(20,10))
df_T_test.plot(x='ds', y='y', ax=ax)
plt.fill_between(od_preds['data']['forecast']['ds'].values, 
                 od_preds['data']['forecast']['yhat_lower'], 
                 od_preds['data']['forecast']['yhat_upper'], alpha=0.2)

In [ ]:
ax=od_preds['data']['forecast'].plot(x='ds', y=['score'], figsize=(20,10))
ax.axhline(y=0, label='threshold', color='red')

In [ ]:
df_kwh_H_mean_outage.iloc[n_prophet:][df_kwh_H_mean_outage.iloc[n_prophet:][value_col].isna()]

In [ ]:
# col_of_interest='value'
# threshold=.9

col_of_interest='returns'
threshold=.99

# col_of_interest='sq_returns'
# threshold=.9

In [ ]:
exclude_fit_min = out_t_end.floor(freq='D')-datetime.timedelta(days=3)
exclude_fit_max = out_t_end.ceil(freq='D')+datetime.timedelta(days=3)
df_kwh_H_mean_outage['2020-06-20 00:00:00':'2020-06-23 00:00:00'][col_of_interest].plot()

In [ ]:
df_train = df_kwh_H_mean_outage[[col_of_interest]].copy()
df_train[exclude_fit_min:exclude_fit_max] = np.nan
df_train = df_train.reset_index()
df_train = df_train.rename(columns={'time_idx':'ds', col_of_interest:'y'})
df_train.loc[df_train['y'].isin([np.inf, -np.inf]), 'y'] = np.nan
df_train.plot(x='ds', y='y', figsize=(20,10))

In [ ]:
df_test = df_kwh_H_mean_outage[exclude_fit_min:exclude_fit_max][col_of_interest].copy()
df_test = df_test.reset_index()
df_test = df_test.rename(columns={'time_idx':'ds', col_of_interest:'y'})
df_test.loc[df_test['y'].isin([np.inf, -np.inf]), 'y'] = np.nan
df_test.plot(x='ds', y='y', figsize=(20,10))

In [ ]:
df_full = df_kwh_H_mean_outage[col_of_interest].copy()
df_full = df_full.reset_index()
df_full = df_full.rename(columns={'time_idx':'ds', col_of_interest:'y'})
df_full.loc[df_full['y'].isin([np.inf, -np.inf]), 'y'] = np.nan
df_full.plot(x='ds', y='y', figsize=(20,10))

In [ ]:
od2 = OutlierProphet(threshold=threshold)
od2.fit(df_train)

In [ ]:
od2_preds = od2.predict(
    df_test,
    return_instance_score=True,
    return_forecast=True
)
ax=od2_preds['data']['forecast'].plot(x='ds', y=['yhat'], figsize=(20,10))
df_test.plot(x='ds', y='y', ax=ax)
df_test[od2_preds['data']['is_outlier']['is_outlier']==1].plot.scatter(x='ds', y='y', ax=ax, color='red', label='Outliers')
plt.fill_between(od2_preds['data']['forecast']['ds'].values, 
                 od2_preds['data']['forecast']['yhat_lower'], 
                 od2_preds['data']['forecast']['yhat_upper'], alpha=0.2)
ax.axvline(out_t_beg, color='red')
ax.axvline(out_t_end, color='lawngreen')

In [ ]:
od2_preds = od2.predict(
    df_full,
    return_instance_score=True,
    return_forecast=True
)
ax=od2_preds['data']['forecast'].plot(x='ds', y=['yhat'], figsize=(20,10))
df_full.plot(x='ds', y='y', ax=ax)
df_full[od2_preds['data']['is_outlier']['is_outlier']==1].plot.scatter(x='ds', y='y', ax=ax, color='red', label='Outliers')
plt.fill_between(od2_preds['data']['forecast']['ds'].values, 
                 od2_preds['data']['forecast']['yhat_lower'], 
                 od2_preds['data']['forecast']['yhat_upper'], alpha=0.2)
ax.axvline(out_t_beg, color='red')
ax.axvline(out_t_end, color='lawngreen')

In [ ]:
import stumpy
from matplotlib.patches import Rectangle

In [ ]:
m=24
col_of_interest=value_col
mp = stumpy.stump(df_kwh_H_mean_outage[col_of_interest], m)

In [ ]:
motif_idx = np.argsort(mp[:, 0])[0]
print(f"The motif is located at index {motif_idx}")

In [ ]:
nearest_neighbor_idx = mp[motif_idx, 1]
print(f"The nearest neighbor is located at index {nearest_neighbor_idx}")

In [ ]:
fig, axs = plt.subplots(2, sharex=True, gridspec_kw={'hspace': 0}, figsize=(20,10))
plt.suptitle('Motif (Pattern) Discovery', fontsize='30')

axs[0].plot(df_kwh_H_mean_outage[col_of_interest].values)
axs[0].set_ylabel('kWh Mean', fontsize='20')
rect = Rectangle((motif_idx, 0), m, 
                 df_kwh_H_mean_outage[~df_kwh_H_mean_outage[col_of_interest].isin([np.inf, -np.inf])][col_of_interest].max(), 
                 facecolor='lightgrey')
axs[0].add_patch(rect)
rect = Rectangle((nearest_neighbor_idx, 0), m, 
                 df_kwh_H_mean_outage[~df_kwh_H_mean_outage[col_of_interest].isin([np.inf, -np.inf])][col_of_interest].max(), 
                 facecolor='lightgrey')
axs[0].add_patch(rect)
axs[1].set_xlabel('Time', fontsize ='20')
axs[1].set_ylabel('Matrix Profile', fontsize='20')
axs[1].axvline(x=motif_idx, linestyle="dashed")
axs[1].axvline(x=nearest_neighbor_idx, linestyle="dashed")
axs[1].plot(mp[:, 0])


In [ ]:
discord_idx = np.argsort(mp[:, 0])[-1]
print(f"The discord is located at index {discord_idx}")

In [ ]:
nearest_neighbor_distance = mp[discord_idx, 0]
print(f"The nearest neighbor subsequence to this discord is {nearest_neighbor_distance} units away")

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing

In [ ]:
df_kwh_H_mean_outage[value_col].plot(figsize=(20,10))

In [ ]:
df_kwh_H_mean_no_outage_cpy = df_kwh_H_mean_no_outage.copy()
df_kwh_H_mean_no_outage_cpy['ExpSmooth'] = ExponentialSmoothing(df_kwh_H_mean_no_outage_cpy[value_col], trend='add', seasonal='add', seasonal_periods=24).fit().fittedvalues

In [ ]:
df_kwh_H_mean_no_outage_cpy.plot(y=[value_col, 'ExpSmooth'], figsize=(20,10))

# fbprophet testing
from https://machinelearningmastery.com/time-series-forecasting-with-prophet-in-python/

In [ ]:
# check prophet version
import fbprophet
# print version number
print('Prophet %s' % fbprophet.__version__)

In [ ]:
# load the car sales dataset
from pandas import read_csv
# load data
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-car-sales.csv'
df = read_csv(path, header=0)
# summarize shape
print(df.shape)
# show first few rows
print(df.head())

In [ ]:
# load and plot the car sales dataset
from pandas import read_csv
from matplotlib import pyplot
# load data
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-car-sales.csv'
df = read_csv(path, header=0)
# plot the time series
df.plot()
pyplot.show()

In [ ]:
# prepare expected column names
df.columns = ['ds', 'y']
df['ds']= pd.to_datetime(df['ds'])

In [ ]:
# fit prophet model on the car sales dataset
from pandas import read_csv
from pandas import to_datetime
from fbprophet import Prophet
# load data
path = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-car-sales.csv'
df = read_csv(path, header=0)
# prepare expected column names
df.columns = ['ds', 'y']
df['ds']= to_datetime(df['ds'])
# define the model
model = Prophet()
# fit the model
model.fit(df)

In [ ]:
# define the period for which we want a prediction
future = list()
for i in range(1, 13):
    date = '1968-%02d' % i
    future.append([date])
future = pd.DataFrame(future)
future.columns = ['ds']
future['ds']= to_datetime(future['ds'])

In [ ]:
# summarize the forecast
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())

In [ ]:
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())
# plot forecast
model.plot(forecast);

In [ ]:
# define the period for which we want a prediction
future = list()
for i in range(1, 13):
    date = '1969-%02d' % i
    future.append([date])
future = pd.DataFrame(future)
future.columns = ['ds']
future['ds']= to_datetime(future['ds'])

In [ ]:
# use the model to make a forecast
forecast = model.predict(future)
# summarize the forecast
print(forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())
# plot forecast
model.plot(forecast)
pyplot.show()

In [ ]:
# create test dataset, remove last 12 months
train = df.drop(df.index[-12:])
print(train.tail())

In [ ]:
from sklearn.metrics import mean_absolute_error
# calculate MAE between expected and predicted values for december
y_true = df['y'][-12:].values
y_pred = forecast['yhat'].values
mae = mean_absolute_error(y_true, y_pred)
print('MAE: %.3f' % mae)

In [ ]:
# plot expected vs actual
pyplot.plot(y_true, label='Actual')
pyplot.plot(y_pred, label='Predicted')
pyplot.legend()
pyplot.show()

In [ ]:
df_T['y'].plot(figsize=(12,5))

In [ ]:
my_model = Prophet()
my_model.fit(df_T)

In [ ]:
df_T

In [ ]:
df_T_last_week = df_T.iloc[-7*24:].copy()
df_T_last_week.drop(columns=['y'], inplace=True)

In [ ]:
df_T_last_week

In [ ]:
my_forecast_last_week = my_model.predict(df_T_last_week)

In [ ]:
print(my_forecast_last_week[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head())

In [ ]:
my_model.plot(my_forecast_last_week);

In [ ]:
df_T.iloc[-7*24:].reset_index()['y'].plot()
my_forecast_last_week['yhat'].plot()

In [ ]:
df_kwh_H_mean_outage[~df_kwh_H_mean_outage['returns'].isin([np.nan, np.inf, -np.inf])].describe()

In [ ]:
trim(df_kwh_H_mean_outage[~df_kwh_H_mean_outage['returns'].isin([np.nan, np.inf, -np.inf])], (0.0, 0.0025), relative=True).max()

In [ ]:
df_kwh_H_mean_outage[~df_kwh_H_mean_outage['returns'].isin([np.nan, np.inf, -np.inf])]['returns'].hist()

In [ ]:
df_kwh_H_mean_outage[~df_kwh_H_mean_outage['returns'].isin([np.nan, np.inf, -np.inf])][value_col].max()

In [ ]:
from scipy.stats.mstats import trim

In [ ]:
tmp = trim(df_kwh_15T_outage['returns'], (0.02, 0.02), relative=True)

In [ ]:
tmp.max()

In [ ]:
tmp_df = df_kwh_15T_outage.copy()
tmp_df = tmp_df[tmp_df['returns'].notna()]
tmp_df = tmp_df[~tmp_df['returns'].isin([np.nan, np.inf, -np.inf])]
for _ in range(1):
    tmp_df, info = GrubbsTest.identify_outliers_in_metric_column(tmp_df, 'returns', include_output_info=True, inplace=False)

In [ ]:
tmp_df['returns'].max()

In [ ]:
df_kwh_15T_mean_outage_cpy = df_kwh_15T_mean_outage.copy()
df_kwh_15T_mean_outage_cpy.loc[df_kwh_15T_mean_outage_cpy['sq_returns'].isin([np.inf, -np.inf]), 'sq_returns'] = np.nan
tmp_lst = sorted(df_kwh_15T_mean_outage_cpy['sq_returns'].values, reverse=True)

n_within_order_of_mag = 10
limit_found = False
i=0
while i<len(tmp_lst) and not limit_found:
    limit = tmp_lst[i]
    ratio = tmp_lst[i]/tmp_lst[i+n_within_order_of_mag]
    if ratio < 10:
        limit_found = True
    i+=1
    
print(limit)

In [ ]:
limit

In [ ]:
i

In [ ]:
tmp_lst[5+10]

In [ ]:
tmp_lst[1]/tmp_lst[1+10]

In [ ]:
ax=df_kwh_15T_mean_outage_cpy['sq_returns'].plot(figsize=(20,5))
ax.set_ylim([0,70000])

In [ ]:
ax=df_kwh_15T_mean_outage['sq_returns'].plot(figsize=(20,5))
plt.title("15T Mean Volatility - Outage (AFTER)", size = 24)
ax.set_ylim([0, limit])

In [ ]:
dfs_vlt_dict['H'][df_key]

In [ ]:
expand_time = datetime.timedelta(days=10)
dfs_vlt_dict['H'][df_agg_key][out_t_beg-expand_time:out_t_end+expand_time].plot(x='endtimeperiod_utc', y=value_col, figsize=(20,5))

In [ ]:
df_vlt_H_mean_cpy = dfs_vlt_dict['H'][df_agg_key].copy()
#df_vlt_H_mean_cpy.loc[df_vlt_H_mean_cpy['value'].isin([np.inf, -np.inf]), 'value'] = np.nan
df_vlt_H_mean_cpy=df_vlt_H_mean_cpy.loc[~df_vlt_H_mean_cpy[value_col].isin([np.nan, np.inf, -np.inf])]
tmp_lst = sorted(df_vlt_H_mean_cpy[value_col].values, reverse=False)
tmp_lst=np.unique(tmp_lst)

n_within_order_of_mag = 10
limit_found = False
i=0
while i<len(tmp_lst)-n_within_order_of_mag and not limit_found:
    limit = tmp_lst[i]
    ratio = tmp_lst[i]/tmp_lst[i+n_within_order_of_mag]
    print(ratio)
    if ratio > 0.1:
        limit_found = True
    i+=1
    
print(limit)

In [ ]:
limit_found

In [ ]:
tmp_lst[1]

In [ ]:
tmp_lst[2:]

In [ ]:
limit

In [ ]:
len(tmp_lst)

In [ ]:
# tmp_lst[27805]

In [ ]:
len(tmp_lst)

In [ ]:
tmp_lst[19388:]

In [ ]:
df_vlt_H_mean_cpy = dfs_vlt_dict['H'][df_agg_key].copy()
#df_vlt_H_mean_cpy.loc[df_vlt_H_mean_cpy['value'].isin([np.inf, -np.inf]), 'value'] = np.nan
df_vlt_H_mean_cpy=df_vlt_H_mean_cpy.loc[~df_vlt_H_mean_cpy[value_col].isin([np.nan, np.inf, -np.inf])]

In [ ]:
df_vlt_H_mean_cpy.shape

In [ ]:
dfs_vlt_dict['H'][df_agg_key].shape

In [ ]:
tmp_lst = sorted(df_vlt_H_mean_cpy[value_col].values, reverse=False)

In [ ]:
tmp_lst

In [ ]:
len(np.unique(tmp_lst))

In [ ]:
df_vlt_H_mean_cpy.groupby(value_col).ngroups

In [ ]:
df_vlt_H_mean_cpy.shape

In [ ]:
fuckn_df = df_kwh_15T[df_kwh_15T['serialnumber']==880693914].copy()
fuckn_df.head()

In [ ]:
fuckn_df['annual_kwh'].unique()

In [ ]:
fuckn_df['value'].sum()

In [ ]:
fuckn_df['2020-01-01 00:00:00':'2020-12-31 23:59:59']['value'].sum()

In [ ]:
fuckn_df['2020-01-01 00:00:00':'2020-12-31 23:59:59']['value'].max()

In [ ]:
df_kwh_15T['prem_nb'].unique()

In [ ]:
df_kwh_15T['aep_derived_uom'].nunique()

In [ ]:
#prem_nb = 105148681
prem_nb = 108248681

In [ ]:
ax=df_kwh_15T[df_kwh_15T['prem_nb']==prem_nb]['value'].rolling(window=35040).sum().plot()
assert(df_kwh_15T[df_kwh_15T['prem_nb']==prem_nb]['annual_kwh'].nunique()==1)
ax.axhline(y=df_kwh_15T[df_kwh_15T['prem_nb']==prem_nb].iloc[0]['annual_kwh'], 
           label='Quoted Annual kWh', color='red')


In [ ]:
df_kwh_15T[df_kwh_15T['prem_nb']==prem_nb]['annual_kwh'].value_counts()

In [ ]:
df_kwh_15T[df_kwh_15T['prem_nb']==prem_nb]['2020-01-01 00:00:00':'2020-12-31 23:59:59']['value'].sum()

In [ ]:
df_kwh_15T[df_kwh_15T['prem_nb']==prem_nb].iloc[0]['annual_kwh']

In [ ]:
df_kwh_15T['prem_nb'].nunique()

In [ ]:
def save_fig(fig, save_dir, save_name):
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)
    save_path = os.path.join(save_dir, save_name)
    fig.savefig(save_path)

In [ ]:
save = False
fig, axs = plt.subplots(2, 3, num=fig_num, figsize=[28, 18])
plt.subplots_adjust(left=0.075, right=0.975, 
                    top=0.95, bottom=0.75, 
                    hspace=0.5)
axs = axs.flatten()
for i,prem_nb in enumerate(df_kwh_15T['prem_nb'].unique()):
    df_kwh_15T[df_kwh_15T['prem_nb']==prem_nb]['value'].rolling(window=35040).sum().plot(ax=axs[i], label='12 month rolling average')
    assert(df_kwh_15T[df_kwh_15T['prem_nb']==prem_nb]['annual_kwh'].nunique()==1)
    axs[i].axhline(y=df_kwh_15T[df_kwh_15T['prem_nb']==prem_nb].iloc[0]['annual_kwh'], 
               label='Quoted Annual kWh', color='red')
    axs[i].legend(fontsize=12)
    axs[i].set_title(label=f'Premise #{prem_nb}', fontsize=25)
    axs[i].set_xlabel(xlabel='Datetime', fontsize=20, x=0.9, y=0.0, ha='right', va='top')
    axs[i].tick_params(axis='x', labelrotation=45, labelsize=15)
    axs[i].tick_params(axis='y', labelsize=15)
if save:
    save_dir = r'C:\Users\s346557\Documents\Presentations\GroupMeetings\20220120\Figures\annual_kwh'
    save_name = 'how_is_annual_kwh_calc.png'
    save_fig(fig, save_dir, save_name)
fig_num += 1

In [ ]:
df_kwh_15T